In [ ]:
# 📦 KÜTÜPHANE KURULUMU
# Tüm gerekli kütüphaneleri kur (ilk çalıştırmada çalıştır)

import subprocess
import sys

print("📦 Gerekli kütüphaneler yükleniyor...\n")

libraries = {
    "torch": "PyTorch (Deep Learning)",
    "torchvision": "TorchVision (Image Models)",
    "pandas": "Pandas (Data Processing)",
    "numpy": "NumPy (Array Operations)",
    "pillow": "Pillow (Image Loading)",
    "tqdm": "TQDM (Progress Bar)",
    "matplotlib": "Matplotlib (Plotting)",
    "seaborn": "Seaborn (Advanced Plots)",
    "scikit-learn": "Scikit-Learn (ML Utils)"
}

print("=" * 70)
print("🎯 KURULACAK KÜTÜPHANELER:")
print("=" * 70)
for i, (lib, desc) in enumerate(libraries.items(), 1):
    print(f"   {i}. {lib:<20} → {desc}")

print("\n" + "=" * 70)
print("⚡ KURULUM BAŞLIYOR...\n")

# Pip upgrade
print("1️⃣ Pip yükseltiliyor...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "-q"])
print("   ✅ Pip güncelleştirildi\n")

# Standart libraries
standard_libs = ["pandas", "pillow", "numpy", "tqdm", "matplotlib", "seaborn", "scikit-learn"]
print("2️⃣ Standart kütüphaneler yükleniyor...")
for lib in standard_libs:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "-q"])
        print(f"   ✅ {lib}")
    except Exception as e:
        print(f"   ⚠️ {lib} kurulumunda hata: {e}")

# PyTorch NIGHTLY (RTX 5080 Desteği - CUDA 12.8)
print("\n3️⃣ PyTorch NIGHTLY (RTX 5080 + CUDA 12.8) yükleniyor...")
print("   ⚠️  Bu biraz uzun sürebilir (~2-3 dakika)")

try:
    # Önce eski PyTorch'u kaldır
    print("   → Eski PyTorch kaldırılıyor...")
    subprocess.call([sys.executable, "-m", "pip", "uninstall", "torch", "torchvision", "-y"], 
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    # PyTorch Nightly + CUDA 12.8 kur (RTX 5080 desteği!)
    print("   → PyTorch Nightly (CUDA 12.8) kuruluyor...")
    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "--pre", "torch", "torchvision",
        "--index-url", "https://download.pytorch.org/whl/nightly/cu128"
    ])
    print("   ✅ PyTorch Nightly + CUDA 12.8 yüklendi (RTX 5080 destekli!)")
except Exception as e:
    print(f"   ❌ PyTorch Nightly kurulumunda hata: {e}")
    print("   → Fallback: CUDA 12.6 deneniyor...")
    try:
        subprocess.check_call([
            sys.executable, "-m", "pip", "install",
            "torch", "torchvision",
            "--index-url", "https://download.pytorch.org/whl/cu126"
        ])
        print("   ⚠️  CUDA 12.6 yüklendi (RTX 5080 tam desteklenmeyebilir)")
    except:
        print("   ❌ CUDA 12.6 de başarısız!")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torch", "torchvision"])
        print("   ⚠️  Standart PyTorch yüklendi")

print("\n" + "=" * 70)
print("✅ KURULUM TAMAMLANDI!\n")

# Kontrol
print("4️⃣ Kontrol ediliyor...\n")
try:
    import torch
    print(f"   ✅ PyTorch: {torch.__version__}")
    print(f"   ✅ CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   ✅ GPU: {torch.cuda.get_device_name(0)}")
    
    import pandas as pd
    print(f"   ✅ Pandas: {pd.__version__}")
    
    import numpy as np
    print(f"   ✅ NumPy: {np.__version__}")
    
    print("\n" + "=" * 70)
    print("🎉 TÜM KÜTÜPHANELER HAZIR!")
    print("=" * 70)
except Exception as e:
    print(f"   ❌ Hata: {e}")
    print("   → Lütfen hata mesajını kontrol et ve tekrar çalıştır")


# 🏗️ Hiyerarşik Mask Detection - FULL VERSION (OPTIMIZED)

## 🚀 YENİ ÖZELLİKLER:
- ✅ Mixed Precision Training (AMP) - %40 daha az VRAM, 2-3x daha hızlı
- ✅ Gradient Accumulation - Büyük batch size simülasyonu
- ✅ Optimize DataLoader - Multi-worker, persistent workers, prefetch
- ✅ Büyük Batch Size - GPU gücünü maksimum kullanma
- ✅ CSV ve Extract aynı konumda - Tek base path
- ✅ PyTorch 2.0 Compile - %20-30 hız artışı
- ✅ Checkpoint Resume - Eğitim devam ettirme

## Konsept:
```
Stage 1: Domain Classification (5 sınıf)
  ├─ URBAN (15 masks)
  ├─ HUMAN_BODY (14 masks)
  ├─ CLOTHING (7 masks)
  ├─ INDOOR (8 masks)
  └─ BACKGROUND (5 masks)

Stage 2: Fine-Grained Mask Detection
  ├─ URBAN Model → building, car, road, ...
  ├─ HUMAN_BODY Model → hair, face, skin, ...
  ├─ CLOTHING Model → Upper-clothes, Pants, ...
  ├─ INDOOR Model → floor, ceiling, chair, ...
  └─ BACKGROUND Model → background, ego vehicle, ...
```

## Coverage: ~77% (Top-50 eşdeğer)

In [ ]:
"""
🔧 HÜCRE 1 - SETUP VE IMPORTS (OPTIMIZED)
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.cuda.amp import autocast, GradScaler  # ✨ YENİ: Mixed Precision
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
import glob
import random
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import json
from collections import defaultdict
from sklearn.metrics import confusion_matrix, classification_report
from utils import *

# ✨ YENİ: GPU Optimizasyonları
cudnn.benchmark = True  # En hızlı conv algoritmasını otomatik seç
cudnn.deterministic = False  # Deterministiklik yerine hız öncelikli
torch.backends.cuda.matmul.allow_tf32 = True  # TF32 precision (A100+ için)
torch.backends.cudnn.allow_tf32 = True
torch.cuda.empty_cache()

print("🔥 GPU ısındırılıyor ve optimize ediliyor...")
dummy = torch.randn(2000, 2000).cuda()  # Daha büyük warm-up
for _ in range(20):  # Daha fazla iterasyon
    _ = torch.matmul(dummy, dummy)
torch.cuda.synchronize()
del dummy
torch.cuda.empty_cache()
print("✅ GPU hazır ve optimize edildi!")

# =============================================
# ⚙️ AYARLAR - OPTIMIZED
# =============================================

NOTEBOOK_NAME = "hierarchical_mask_detection"

# ✨ RTX 5080 İÇİN OPTİMİZE AYARLAR (16GB VRAM, 24C/32T CPU, 64GB RAM)
BATCH_SIZE = 64  # RTX 5080 16GB VRAM için optimal

# ✨ Gradient Accumulation (Effective batch = BATCH_SIZE * ACCUMULATION_STEPS)
ACCUMULATION_STEPS = 2  # Effective batch = 64 * 2 = 128

# ✨ DataLoader İşçi Sayısı (Windows için optimize)
# Windows'ta multi-worker sorunlu olabilir, 0 kullan (CPU-GPU dengesi için)
NUM_WORKERS = 0  # Windows'ta 0 önerilir (CPU'yu meşgul etmez)

# Eğitim ayarları
NUM_EPOCHS_STAGE1 = 6  # Domain classifier
NUM_EPOCHS_STAGE2 = 5  # Domain-specific models
LEARNING_RATE = 0.001
EARLY_STOP_PATIENCE = 8
IMG_SIZE = 224
VAL_SPLIT = 0.2

# ✨ YENİ: CSV ve Extract aynı konumda
CSV_BASE = r"C:\AI_DATA\SEMI_TRUTHS\inpainting"  # ⚠️ CSV dosyalarının konumu
EXTRACTED_BASE = r"C:\AI_DATA\SEMI_TRUTHS_extracted\inpainting"  # ⚠️ Extract edilmiş görsellerin konumu

# ✨ Mixed Precision Ayarları
USE_AMP = True  # Automatic Mixed Precision (FP16) - MUTLAKA True (VRAM %40 azaltır)
USE_COMPILE = False  # PyTorch 2.0+ compile - RTX 5080 tam desteklenene kadar False

# ✨ YENİ: Checkpoint/Resume
RESUME_TRAINING = False  # Eğitimi devam ettir
SAVE_EVERY_N_EPOCHS = 1  # Her N epoch'ta checkpoint kaydet

# ═══════════════════════════════════════════════════════════════════════
# 🧪 TEST AYARLARI - KOLAY AYARLANABILIR
# ═══════════════════════════════════════════════════════════════════════

# 📊 TEST VERİSİ KULLANIM ORANI (0.0 - 1.0 arası)
# Örnek: 0.25 = %25, 0.5 = %50, 1.0 = %100 (tüm veri)
TEST_DATA_RATIO = 0.25  # 🔧 Buradan kolayca ayarla!

# 🔄 MULTI-EXPERT ROUTING (URBAN/INDOOR/BACKGROUND karışıklığını çözmek için)
USE_MULTI_EXPERT_ROUTING = True  # Multi-expert routing aktif mi?
CONFUSING_DOMAINS = ['URBAN', 'INDOOR', 'BACKGROUND']  # Karışan domain'ler
ROUTING_THRESHOLD = 0.90  # Bu threshold'un altında multi-expert devreye girer

# ⚖️ HYBRID SCORING PARAMETRELERİ
# Score = α × P(domain) + (1-α) × P(mask) × confidence_factor
USE_DYNAMIC_ALPHA = True  # ✨ YENİ: Domain'e göre dinamik alpha kullan
HYBRID_ALPHA = 0.3  # 0.3 = Stage2'ye daha fazla güven (USE_DYNAMIC_ALPHA=False ise kullanılır)
                    # 0.5 = Eşit ağırlık
                    # 0.7 = Stage1'e daha fazla güven

device = torch.device("cuda")

print("="*70)
print("🎮 HİYERARŞİK MASK DETECTION - RTX 5080 OPTIMIZED")
print("="*70)
print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"🔧 CUDA Capability: {torch.cuda.get_device_capability(0)}")
print(f"🔥 CUDA Cores: ~{torch.cuda.get_device_properties(0).multi_processor_count * 128}")
print("─"*70)
print(f"📊 PERFORMANS AYARLARI (RTX 5080 Optimized):")
print(f"   • Batch Size: {BATCH_SIZE}")
print(f"   • Gradient Accumulation: {ACCUMULATION_STEPS} (Effective Batch: {BATCH_SIZE * ACCUMULATION_STEPS})")
print(f"   • DataLoader Workers: {NUM_WORKERS} / 32 threads")
print(f"   • Mixed Precision (AMP): {'✅ Aktif (VRAM tasarruf: %40)' if USE_AMP else '❌ Kapalı'}")
print(f"   • PyTorch Compile: {'✅ Aktif' if USE_COMPILE else '❌ Kapalı (RTX 5080 uyumluluk bekleniyor)'}")
print("─"*70)
print(f"🎯 EĞİTİM AYARLARI:")
print(f"   • Stage 1 Epochs: {NUM_EPOCHS_STAGE1}")
print(f"   • Stage 2 Epochs: {NUM_EPOCHS_STAGE2}")
print(f"   • Learning Rate: {LEARNING_RATE}")
print(f"   • Early Stop Patience: {EARLY_STOP_PATIENCE}")
print("─"*70)
print(f"📂 VERİ KONUMLARI:")
print(f"   • CSV Base: {CSV_BASE}")
print(f"   • Extracted Base: {EXTRACTED_BASE}")
print(f"   ⚠️  Konumları güncelleyin!")
print("─"*70)
print(f"⚡ TAHMİNİ PERFORMANS (RTX 5080):")
print(f"   • Epoch süresi: ~2-3 dakika (80K görüntü)")
print(f"   • GPU kullanımı: %95+")
print(f"   • VRAM kullanımı: ~9-10GB / 16GB")
print(f"   • Throughput: ~800-1000 img/sec")
print("─"*70)
print(f"🧪 TEST AYARLARI:")
print(f"   • Test Veri Oranı: {TEST_DATA_RATIO*100:.0f}%")
print(f"   • Multi-Expert Routing: {'✅ Aktif' if USE_MULTI_EXPERT_ROUTING else '❌ Kapalı'}")
if USE_MULTI_EXPERT_ROUTING:
    print(f"   • Karışan Domain'ler: {CONFUSING_DOMAINS}")
    print(f"   • Routing Threshold: {ROUTING_THRESHOLD}")
    if USE_DYNAMIC_ALPHA:
        print(f"   • Hybrid Alpha: {'✅ Dinamik (Domain bazlı)'}")
    else:
        print(f"   • Hybrid Alpha: {HYBRID_ALPHA} (Stage1 ağırlığı - Sabit)")
print("="*70)

# ✨ YENİ: Mixed Precision Scaler
scaler = GradScaler(enabled=USE_AMP)
print("\n✅ GradScaler hazır (Mixed Precision)")

In [ ]:
"""
📋 HÜCRE 2 - DOMAIN KATEGORİLERİ
"""

print("\n" + "="*70)
print("📋 DOMAIN KATEGORİLERİ")
print("="*70)

DOMAIN_CATEGORIES = {
    'URBAN': [
        'building', 'car', 'road', 'sidewalk', 'vegetation', 'sky', 
        'wall', 'window', 'door', 'tree', 'trees', 'pole', 'fence', 
        'parking', 'terrain'
    ],
    
    'HUMAN_BODY': [
        'hair', 'face', 'skin', 'neck', 'nose', 'Right-arm', 'Left-arm',
        'Right-leg', 'Left-leg', 'left_ear', 'right_ear', 'mouth', 
        'upper_lip', 'lower_lip'
    ],
    
    'CLOTHING': [
        'Upper-clothes', 'Pants', 'skirt', 'dress', 'hat', 'bag', 'cloth'
    ],
    
    'INDOOR': [
        'floor', 'ceiling', 'chair', 'table', 'seat', 'column', 'ground', 'grass'
    ],
    
    'BACKGROUND': [
        'background', 'out of roi', 'ego vehicle', 'rectification border', 'static'
    ]
}

DOMAIN_NAMES = list(DOMAIN_CATEGORIES.keys())
NUM_DOMAINS = len(DOMAIN_NAMES)

print(f"\n✅ {NUM_DOMAINS} Domain Tanımlandı:")
total_masks = 0
for i, (domain, masks) in enumerate(DOMAIN_CATEGORIES.items()):
    print(f"   [{i}] {domain:<15} → {len(masks):2d} masks")
    total_masks += len(masks)

print(f"\n   TOPLAM: {total_masks} mask türü")

def get_domain(mask_name):
    for domain, masks in DOMAIN_CATEGORIES.items():
        if mask_name in masks:
            return domain
    return None

print("\n✅ get_domain() fonksiyonu tanımlandı")

In [ ]:
"""
📊 HÜCRE 3 - CSV YÜKLEME VE DOMAIN ETİKETLEME
"""

print("\n" + "="*70)
print("📊 CSV YÜKLEMESİ")
print("="*70)

csv_files = glob.glob(os.path.join(CSV_BASE, "**", "*_meta.csv"), recursive=True)
print(f"\n✅ Bulunan CSV: {len(csv_files)} dosya")

if len(csv_files) == 0:
    raise FileNotFoundError(f"❌ CSV dosyası bulunamadı: {CSV_BASE}")

all_dfs = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    csv_dir = os.path.dirname(csv_file)
    parent_folder = os.path.basename(os.path.dirname(csv_dir))
    model_name = os.path.basename(csv_dir)
    df['parent_dataset'] = parent_folder
    df['model'] = model_name
    df['dataset'] = f"{parent_folder}_{model_name}"
    all_dfs.append(df)
    print(f"   ✅ {parent_folder}/{model_name}: {len(df):,} satır")

df_combined = pd.concat(all_dfs, ignore_index=True)
print(f"\n✅ Toplam: {len(df_combined):,} görsel")

# Domain etiketleme
df_combined['domain'] = df_combined['mask_name'].apply(get_domain)
df_with_domain = df_combined[df_combined['domain'].notna()].copy()

print(f"\n📊 Domain Dağılımı:")
domain_counts = df_with_domain['domain'].value_counts()
for domain in DOMAIN_NAMES:
    count = domain_counts.get(domain, 0)
    ratio = (count / len(df_with_domain)) * 100
    print(f"   {domain:<15}: {count:>6,} ({ratio:>5.1f}%)")

print(f"\n✅ Domain etiketlendi: {len(df_with_domain):,} / {len(df_combined):,} görsel")

In [ ]:
"""
🏗️ HÜCRE 4 - MODEL MİMARİLERİ
"""

print("\n" + "="*70)
print("🏗️ MODEL MİMARİLERİ")
print("="*70)

class DomainClassifier(nn.Module):
    """Stage 1: Domain sınıflandırıcı"""
    
    def __init__(self, num_domains=5, dropout_rate=0.3):
        super(DomainClassifier, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        backbone_out = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        
        self.head = nn.Sequential(
            nn.Linear(backbone_out, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_domains)
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

class DomainMaskDetector(nn.Module):
    """Stage 2: Domain-specific mask detector"""
    
    def __init__(self, num_masks, dropout_rate=0.3):
        super(DomainMaskDetector, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        backbone_out = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        
        self.head = nn.Sequential(
            nn.Linear(backbone_out, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_masks)
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

print("\n✅ DomainClassifier tanımlandı")
print("✅ DomainMaskDetector tanımlandı")

In [ ]:
"""
📂 HÜCRE 6 - STAGE 1 DATASET (Domain Classification) - RECURSIVE FIX
"""

print("\n" + "="*70)
print("📂 STAGE 1 DATASET")
print("="*70)

# Domain label mapping
domain_to_label = {domain: idx for idx, domain in enumerate(DOMAIN_NAMES)}
label_to_domain = {idx: domain for domain, idx in domain_to_label.items()}

print("\nDomain → Label:")
for domain, label in domain_to_label.items():
    print(f"   [{label}] {domain}")

# Etiket sözlüğü
labels_dict_by_dataset = {}
for _, row in df_with_domain.iterrows():
    img_id = str(row['perturbed_img_id'])
    domain = row['domain']
    label = domain_to_label[domain]
    dataset_key = row['dataset']
    
    if dataset_key not in labels_dict_by_dataset:
        labels_dict_by_dataset[dataset_key] = {}
    
    for ext in ['', '.png', '.jpg', '.jpeg']:
        labels_dict_by_dataset[dataset_key][img_id + ext] = label

print(f"\n✅ Etiket dictionary oluşturuldu: {len(labels_dict_by_dataset)} dataset")

# =============================================
# 🔍 RECURSIVE DOSYA TARAMA (os.walk kullanarak)
# =============================================

print("\n📂 Extract edilmiş dosyalar taranıyor (recursive)...")

all_image_paths = []
all_labels = []
found_by_dataset = {}

# Her dataset klasörünü tara
for parent_dataset in os.listdir(EXTRACTED_BASE):
    parent_path = os.path.join(EXTRACTED_BASE, parent_dataset)
    if not os.path.isdir(parent_path):
        continue
    
    for model_name in os.listdir(parent_path):
        model_path = os.path.join(parent_path, model_name)
        if not os.path.isdir(model_path):
            continue
        
        dataset_key = f"{parent_dataset}_{model_name}"
        
        # Bu dataset için etiket sözlüğü var mı?
        if dataset_key not in labels_dict_by_dataset:
            continue
        
        labels_dict = labels_dict_by_dataset[dataset_key]
        found_by_dataset[dataset_key] = 0
        
        print(f"\n🔍 {dataset_key}")
        print(f"   Klasör: {model_path}")
        
        # 🔑 核心: os.walk ile recursive tara (alt klasörler dahil)
        for root, dirs, files in os.walk(model_path):
            for file in files:
                if file.endswith(('.png', '.jpg', '.jpeg')):
                    # Dosya adı varyasyonları
                    base_name = file
                    name_without_ext = os.path.splitext(file)[0]
                    
                    label = None
                    
                    # Farklı adlarla dene
                    for name in [base_name, name_without_ext,
                               name_without_ext + '.png',
                               name_without_ext + '.jpg',
                               name_without_ext + '.jpeg']:
                        if name in labels_dict:
                            label = labels_dict[name]
                            break
                    
                    if label is not None:
                        full_path = os.path.join(root, file)
                        all_image_paths.append(full_path)
                        all_labels.append(label)
                        found_by_dataset[dataset_key] += 1
        
        if dataset_key in found_by_dataset:
            print(f"   ✅ {found_by_dataset[dataset_key]:,} görüntü bulundu")

print(f"\n{'='*70}")
print(f"✅ TOPLAM: {len(all_image_paths):,} görüntü bulundu")
print(f"{'='*70}")

if len(all_image_paths) == 0:
    print("\n❌ HATA: Hiç görüntü bulunamadı!")
    print("\n📋 Bulunan dataset'ler:")
    for dataset_key in labels_dict_by_dataset.keys():
        print(f"   - {dataset_key}: {len(labels_dict_by_dataset[dataset_key]):,} etiket")
    raise FileNotFoundError("Görüntü dosyaları bulunamadı!")

# Dataset bazında istatistik
print(f"\n📊 Dataset Bazında Bulunan:")
for dataset_key in sorted(found_by_dataset.keys()):
    expected = len([k for k in labels_dict_by_dataset[dataset_key].keys() if not k.endswith(('.png', '.jpg', '.jpeg'))])
    found = found_by_dataset[dataset_key]
    ratio = (found / expected * 100) if expected > 0 else 0
    print(f"   {dataset_key:<35}: {found:>6,} / {expected:>6,} ({ratio:>5.1f}%)")

# Dataset class
class DomainDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        try:
            img = Image.open(self.image_paths[idx]).convert('RGB')
            if self.transform:
                img = self.transform(img)
            return img, self.labels[idx]
        except Exception as e:
            # Hata durumunda rastgele başka örnek döndür
            return self.__getitem__(random.randint(0, len(self) - 1))

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Train/Val split
indices = list(range(len(all_image_paths)))
random.seed(42)
random.shuffle(indices)

split_idx = int(len(indices) * (1 - VAL_SPLIT))
train_indices = indices[:split_idx]
val_indices = indices[split_idx:]

train_paths_s1 = [all_image_paths[i] for i in train_indices]
train_labels_s1 = [all_labels[i] for i in train_indices]
val_paths_s1 = [all_image_paths[i] for i in val_indices]
val_labels_s1 = [all_labels[i] for i in val_indices]

print(f"\n✅ Train/Val Split:")
print(f"   Train: {len(train_paths_s1):,} görüntü")
print(f"   Val: {len(val_paths_s1):,} görüntü")

# 🔒 STAGE 1 val_indices'i KORU (Cell 8'de override edilmesin!)
stage1_val_indices = val_indices.copy()
stage1_all_image_paths = all_image_paths.copy()
stage1_all_labels = all_labels.copy()

print(f"\n🔒 Stage 1 validation set korundu:")
print(f"   stage1_val_indices: {len(stage1_val_indices):,} örnek")
print(f"   stage1_all_image_paths: {len(stage1_all_image_paths):,} dosya")

# ✨ OPTIMIZED DataLoaders
train_dataset_s1 = DomainDataset(train_paths_s1, train_labels_s1, train_transform)
val_dataset_s1 = DomainDataset(val_paths_s1, val_labels_s1, val_transform)

train_loader_s1 = DataLoader(
    train_dataset_s1, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS,  # ✨ Multi-threaded loading
    pin_memory=True, 
    drop_last=True,
    persistent_workers=True if NUM_WORKERS > 0 else False,  # ✨ Worker'ları canlı tut
    prefetch_factor=2 if NUM_WORKERS > 0 else None  # ✨ Önden 2 batch yükle
)

val_loader_s1 = DataLoader(
    val_dataset_s1, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=NUM_WORKERS,  # ✨ Multi-threaded loading
    pin_memory=True, 
    drop_last=False,
    persistent_workers=True if NUM_WORKERS > 0 else False,  # ✨ Worker'ları canlı tut
    prefetch_factor=2 if NUM_WORKERS > 0 else None  # ✨ Önden 2 batch yükle
)

print(f"\n✅ OPTIMIZED DataLoader'lar oluşturuldu:")
print(f"   Train batches: {len(train_loader_s1)}")
print(f"   Val batches: {len(val_loader_s1)}")
print(f"   Workers: {NUM_WORKERS}")
print(f"   Persistent Workers: {True if NUM_WORKERS > 0 else False}")

print("\n" + "="*70)
print("✅ STAGE 1 DATASET HAZIR")
print("="*70)

In [ ]:
"""
📂 HÜCRE 8 - STAGE 2 DATASET HAZIRLIĞI (Domain-Specific Mask Detection)
Her domain için ayrı dataset ve DataLoader hazırla
"""

print("\n" + "="*70)
print("📂 STAGE 2 DATASET HAZIRLIĞI")
print("="*70)

# =============================================
# 1️⃣ HER DOMAIN İÇİN MASK LABEL MAPPINGS
# =============================================

print("\n1️⃣ Domain-specific mask mappings oluşturuluyor...")

domain_mask_mappings = {}

for domain, masks in DOMAIN_CATEGORIES.items():
    domain_mask_mappings[domain] = {
        'masks': masks,
        'mask_to_label': {mask: idx for idx, mask in enumerate(masks)},
        'label_to_mask': {idx: mask for idx, mask in enumerate(masks)},
        'num_masks': len(masks)
    }

print("\n📊 Domain Mask Counts:")
for domain, info in domain_mask_mappings.items():
    print(f"   {domain:<15}: {info['num_masks']:>2} masks")

# =============================================
# 2️⃣ HER DOMAIN İÇİN DATASET FİLTRELE
# =============================================

print("\n2️⃣ Her domain için dataset filtreleniyor...")

# Dataset class tanımla (Hücre 6'daki ile aynı)
class DomainDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        try:
            img = Image.open(self.image_paths[idx]).convert('RGB')
            if self.transform:
                img = self.transform(img)
            return img, self.labels[idx]
        except Exception as e:
            # Hata durumunda rastgele başka örnek döndür
            return self.__getitem__(random.randint(0, len(self) - 1))

# Transforms (Hücre 6'daki ile aynı)
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Her domain için etiket sözlükleri oluştur
domain_labels_dicts = {}

for domain in DOMAIN_NAMES:
    print(f"\n🔍 {domain} için etiketler hazırlanıyor...")
    
    domain_masks = domain_mask_mappings[domain]['masks']
    mask_to_label = domain_mask_mappings[domain]['mask_to_label']
    
    # Bu domain'e ait mask'lara sahip satırları filtrele
    domain_df = df_with_domain[
        (df_with_domain['domain'] == domain) &
        (df_with_domain['mask_name'].isin(domain_masks))
    ].copy()
    
    print(f"   CSV'de {len(domain_df):,} örnek bulundu")
    
    # Etiket sözlüğü oluştur (dataset bazında)
    labels_dict_by_dataset = {}
    
    for _, row in domain_df.iterrows():
        img_id = str(row['perturbed_img_id'])
        mask_name = row['mask_name']
        label = mask_to_label[mask_name]
        dataset_key = row['dataset']
        
        if dataset_key not in labels_dict_by_dataset:
            labels_dict_by_dataset[dataset_key] = {}
        
        # Farklı uzantılarla denemeler
        for ext in ['', '.png', '.jpg', '.jpeg']:
            labels_dict_by_dataset[dataset_key][img_id + ext] = label
    
    domain_labels_dicts[domain] = labels_dict_by_dataset

# =============================================
# 3️⃣ DOSYA TARAMA (RECURSIVE os.walk)
# =============================================

print("\n3️⃣ Dosyalar taranıyor (recursive)...")

domain_image_data = {}

for domain in DOMAIN_NAMES:
    print(f"\n📂 {domain}")
    
    labels_dict_by_dataset = domain_labels_dicts[domain]
    
    all_image_paths = []
    all_labels = []
    found_by_dataset = {}
    
    # Her dataset klasörünü tara
    for parent_dataset in os.listdir(EXTRACTED_BASE):
        parent_path = os.path.join(EXTRACTED_BASE, parent_dataset)
        if not os.path.isdir(parent_path):
            continue
        
        for model_name in os.listdir(parent_path):
            model_path = os.path.join(parent_path, model_name)
            if not os.path.isdir(model_path):
                continue
            
            dataset_key = f"{parent_dataset}_{model_name}"
            
            # Bu dataset için etiket sözlüğü var mı?
            if dataset_key not in labels_dict_by_dataset:
                continue
            
            labels_dict = labels_dict_by_dataset[dataset_key]
            found_by_dataset[dataset_key] = 0
            
            # 🔑 RECURSIVE TARAMA (os.walk)
            for root, dirs, files in os.walk(model_path):
                for file in files:
                    if file.endswith(('.png', '.jpg', '.jpeg')):
                        # Dosya adı varyasyonları
                        base_name = file
                        name_without_ext = os.path.splitext(file)[0]
                        
                        label = None
                        
                        # Farklı adlarla dene
                        for name in [base_name, name_without_ext,
                                   name_without_ext + '.png',
                                   name_without_ext + '.jpg',
                                   name_without_ext + '.jpeg']:
                            if name in labels_dict:
                                label = labels_dict[name]
                                break
                        
                        if label is not None:
                            full_path = os.path.join(root, file)
                            all_image_paths.append(full_path)
                            all_labels.append(label)
                            found_by_dataset[dataset_key] += 1
    
    # Sonuçları kaydet
    domain_image_data[domain] = {
        'paths': all_image_paths,
        'labels': all_labels,
        'found_by_dataset': found_by_dataset
    }
    
    print(f"   ✅ Toplam: {len(all_image_paths):,} görüntü bulundu")
    
    if len(found_by_dataset) > 0:
        for dataset_key, count in found_by_dataset.items():
            print(f"      • {dataset_key}: {count:,}")

# =============================================
# 4️⃣ TRAIN/VAL SPLIT VE DATALOADER'LAR
# =============================================

print("\n4️⃣ Train/Val split ve DataLoader'lar oluşturuluyor...")

domain_datasets = {}
domain_loaders = {}

for domain in DOMAIN_NAMES:
    print(f"\n📊 {domain}")
    
    image_data = domain_image_data[domain]
    all_paths = image_data['paths']
    all_labels = image_data['labels']
    
    if len(all_paths) == 0:
        print(f"   ⚠️  Görüntü bulunamadı, atlanıyor")
        continue
    
    # Train/Val split
    indices = list(range(len(all_paths)))
    random.seed(42)
    random.shuffle(indices)
    
    split_idx = int(len(indices) * (1 - VAL_SPLIT))
    train_indices = indices[:split_idx]
    val_indices = indices[split_idx:]
    
    train_paths = [all_paths[i] for i in train_indices]
    train_labels = [all_labels[i] for i in train_indices]
    val_paths = [all_paths[i] for i in val_indices]
    val_labels = [all_labels[i] for i in val_indices]
    
    print(f"   Train: {len(train_paths):,} | Val: {len(val_paths):,}")
    
    # Datasets
    train_dataset = DomainDataset(train_paths, train_labels, train_transform)
    val_dataset = DomainDataset(val_paths, val_labels, val_transform)
    
    # ✨ OPTIMIZED DataLoaders (Stage 2)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True,
        num_workers=NUM_WORKERS,  # ✨ Multi-threaded loading
        pin_memory=True, 
        drop_last=True,
        persistent_workers=True if NUM_WORKERS > 0 else False,  # ✨ Worker'ları canlı tut
        prefetch_factor=2 if NUM_WORKERS > 0 else None  # ✨ Önden 2 batch yükle
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False,
        num_workers=NUM_WORKERS,  # ✨ Multi-threaded loading
        pin_memory=True, 
        drop_last=False,
        persistent_workers=True if NUM_WORKERS > 0 else False,  # ✨ Worker'ları canlı tut
        prefetch_factor=2 if NUM_WORKERS > 0 else None  # ✨ Önden 2 batch yükle
    )
    
    # Kaydet
    domain_datasets[domain] = {
        'train': train_dataset,
        'val': val_dataset
    }
    
    domain_loaders[domain] = {
        'train': train_loader,
        'val': val_loader
    }
    
    print(f"   ✅ Train batches: {len(train_loader)} | Val batches: {len(val_loader)}")

print("\n" + "="*70)
print("✅ STAGE 2 DATASET HAZIR")
print("="*70)

# Özet
print(f"\n📊 Özet:")
print(f"   Hazır domain sayısı: {len(domain_loaders)}")
for domain in domain_loaders.keys():
    train_size = len(domain_datasets[domain]['train'])
    val_size = len(domain_datasets[domain]['val'])
    print(f"   • {domain:<15}: {train_size:>6,} train + {val_size:>5,} val")

# 🔬 KAPSAMLI HİYERARŞİK VALİDATİON TEST SİSTEMİ

## Bu bölümde:
1. **HierarchicalDataset**: Hem domain hem mask label'ı içeren dataset
2. **Validation Loader Oluşturma**: CSV'den mask mapping ile
3. **Stage 1 → Stage 2 Pipeline Test**: Tüm validation verisi için
4. **Hata Analizi**: Hangi verilerin nerede yanlış sınıflandırıldığı
5. **Profesyonel Görselleştirmeler**:
   - Confusion Matrices (Stage 1 ve Stage 2)
   - Error Flow Sankey Diagram
   - Per-Domain Accuracy Charts
   - Confidence Distribution Plots
   - Misclassified Sample Image Grids


In [ ]:
"""
🔬 HÜCRE 18 - HİYERARŞİK DATASET VE HELPER FUNCTIONS
Hem domain hem mask label'ını içeren dataset sınıfı
"""

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import pandas as pd
import os
import json
import random
from collections import defaultdict
from tqdm.auto import tqdm

print("\n" + "="*70)
print("🔬 HİYERARŞİK VALİDASYON TEST SİSTEMİ")
print("="*70)

# =============================================
# 📦 HİYERARŞİK DATASET CLASS
# =============================================

class HierarchicalValidationDataset(Dataset):
    """
    Hem domain hem mask label'ını döndüren dataset.
    Stage 1 ve Stage 2 testleri için gerekli tüm bilgileri içerir.
    """
    
    def __init__(self, image_paths, domain_labels, mask_names, mask_labels, 
                 domain_names_list, transform=None):
        """
        Args:
            image_paths: List of image file paths
            domain_labels: List of domain label indices (0-4)
            mask_names: List of actual mask names (string)
            mask_labels: List of mask label indices (domain-specific)
            domain_names_list: List of domain name strings (for reference)
            transform: Image transforms
        """
        self.image_paths = image_paths
        self.domain_labels = domain_labels
        self.mask_names = mask_names
        self.mask_labels = mask_labels
        self.domain_names_list = domain_names_list
        self.transform = transform
        
        # Validate lengths
        assert len(image_paths) == len(domain_labels) == len(mask_names) == len(mask_labels), \
            f"Length mismatch! paths:{len(image_paths)}, domains:{len(domain_labels)}, " \
            f"mask_names:{len(mask_names)}, mask_labels:{len(mask_labels)}"
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        try:
            img = Image.open(self.image_paths[idx]).convert('RGB')
            if self.transform:
                img = self.transform(img)
            
            domain_label = self.domain_labels[idx]
            mask_name = self.mask_names[idx]
            mask_label = self.mask_labels[idx]
            
            return img, domain_label, mask_label, idx  # idx for tracking
            
        except Exception as e:
            print(f"Error loading {self.image_paths[idx]}: {e}")
            # Fallback to random valid sample
            return self.__getitem__(random.randint(0, len(self) - 1))

# =============================================
# 🛠️ HELPER FUNCTIONS
# =============================================

def load_hierarchical_config(config_path):
    """Hierarchical config JSON'ı yükle"""
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    return config

def create_stage2_mask_mappings(config):
    """Stage 2 mask mappings'i oluştur"""
    mappings = {}
    for domain_name, domain_info in config['stage2'].items():
        mappings[domain_name] = {
            'mask_to_label': domain_info['mask_to_label'],
            'label_to_mask': {int(k): v for k, v in domain_info['label_to_mask'].items()},
            'num_masks': domain_info['num_masks']
        }
    return mappings

def get_mask_label_for_domain(mask_name, domain_name, stage2_mappings):
    """Belirli bir domain için mask label'ı döndür"""
    if domain_name not in stage2_mappings:
        return None
    
    mapping = stage2_mappings[domain_name]
    if mask_name in mapping['mask_to_label']:
        return mapping['mask_to_label'][mask_name]
    return None

print("✅ HierarchicalValidationDataset sınıfı tanımlandı")
print("✅ Helper functions tanımlandı")


In [ ]:
"""
📦 HÜCRE 19 - HİYERARŞİK VALİDASYON LOADER OLUŞTURMA
CSV'den mask bilgilerini alarak hierarchical validation loader hazırla

NOT: Bu hücre df_with_domain, all_image_paths, all_labels, val_indices 
değişkenlerinin tanımlı olmasını gerektirir (Cell 3 ve Cell 6)
"""

import re  # regex için

print("\n" + "="*70)
print("📦 HİYERARŞİK VALİDASYON LOADER OLUŞTURMA")
print("="*70)

# =============================================
# 0️⃣ GEREKLİ DEĞİŞKENLERİ KONTROL ET
# =============================================

print("\n0️⃣ Gerekli değişkenler kontrol ediliyor...")

required_vars = ['df_with_domain', 'stage1_all_image_paths', 'stage1_all_labels', 'stage1_val_indices', 
                 'domain_to_label', 'label_to_domain', 'DOMAIN_NAMES', 'DOMAIN_CATEGORIES']
missing_vars = [v for v in required_vars if v not in globals()]

if missing_vars:
    print(f"   ❌ Eksik değişkenler: {missing_vars}")
    print("   ⚠️  Önce Cell 1-6'yı çalıştırın!")
    raise NameError(f"Eksik değişkenler: {missing_vars}")
else:
    print(f"   ✅ Tüm gerekli değişkenler mevcut")
    print(f"      df_with_domain: {len(df_with_domain):,} kayıt")
    print(f"      stage1_all_image_paths: {len(stage1_all_image_paths):,} dosya")
    print(f"      stage1_val_indices: {len(stage1_val_indices):,} örnek")
    
    # 🔍 BOYUT KONTROLÜ
    if len(df_with_domain) < 100000:
        raise ValueError(f"\n❌ df_with_domain çok küçük! ({len(df_with_domain):,})\n"
                         f"   Beklenen: ~121,000 → Cell 3'ü çalıştırın!")
    
    if len(stage1_all_image_paths) < 50000:
        raise ValueError(f"\n❌ stage1_all_image_paths çok küçük! ({len(stage1_all_image_paths):,})\n"
                         f"   Beklenen: ~81,000 → Cell 6'yı çalıştırın!")
    
    if len(stage1_val_indices) < 10000:
        raise ValueError(f"\n❌ stage1_val_indices çok küçük! ({len(stage1_val_indices):,})\n"
                         f"   Beklenen: ~16,000 → Cell 6'yı çalıştırın!")
    
    print(f"      ✅ Boyut kontrolleri PASSED!")

# =============================================
# 1️⃣ CONFIG VE MAPPINGS YÜKLE
# =============================================

print("\n1️⃣ Config ve mappings yükleniyor...")

config_path = os.path.join("eğitim_sonuçları", NOTEBOOK_NAME, "veriler", "hierarchical_config.json")

if not os.path.exists(config_path):
    raise FileNotFoundError(f"Config bulunamadı: {config_path}")

hierarchical_config = load_hierarchical_config(config_path)
stage2_mappings = create_stage2_mask_mappings(hierarchical_config)

print(f"   ✅ Config yüklendi: {config_path}")
print(f"   Stage 1 best accuracy: {hierarchical_config['stage1']['best_val_acc']*100:.2f}%")

for domain, info in stage2_mappings.items():
    print(f"   Stage 2 {domain}: {info['num_masks']} masks")

# =============================================
# 2️⃣ IMAGE PATH → MASK NAME MAPPING OLUŞTUR
# =============================================

print("\n2️⃣ Image → Mask mapping oluşturuluyor...")

# df_with_domain zaten notebook'ta tanımlı (Cell 3'ten)
# Dataset bazlı mask dictionary oluştur - perturbed_img_id → mask_name
mask_dict_by_dataset = {}

for _, row in df_with_domain.iterrows():
    img_id = str(row['perturbed_img_id'])
    mask_name = row['mask_name']
    dataset_key = row['dataset']
    
    if dataset_key not in mask_dict_by_dataset:
        mask_dict_by_dataset[dataset_key] = {}
    
    # Farklı key formatları için (Cell 6'daki mantıkla aynı)
    for ext in ['', '.png', '.jpg', '.jpeg']:
        mask_dict_by_dataset[dataset_key][img_id + ext] = mask_name

print(f"   ✅ {len(mask_dict_by_dataset)} dataset için mask mapping oluşturuldu")
for ds, masks in mask_dict_by_dataset.items():
    unique_masks = len(set(masks.values()))
    print(f"      {ds}: {len(masks)//4:,} image, {unique_masks} unique mask")

# =============================================
# 3️⃣ VALİDATION SET İÇİN LABEL'LARI HAZIRLA
# =============================================

print("\n3️⃣ TEST VERİSİ HAZIRLANIYOR")

# ═══════════════════════════════════════════════════════════════════════
# 📊 TEST_DATA_RATIO ile test edilecek örnek sayısını hesapla
# Bu değişken Cell 2'de (Setup) tanımlı
# ═══════════════════════════════════════════════════════════════════════
target_count = int(len(stage1_all_image_paths) * TEST_DATA_RATIO)
target_count = max(1000, target_count)  # Minimum 1000 örnek

print(f"   📊 TEST_DATA_RATIO: {TEST_DATA_RATIO*100:.0f}%")
print(f"   📊 Hedef örnek sayısı: {target_count:,} / {len(stage1_all_image_paths):,} görsel")
print(f"   ⚠️  NOT: Bazı örnekler atlanabilir (mask bulunamazsa)")
print(f"   💡 Oranı değiştirmek için Cell 2'de TEST_DATA_RATIO değişkenini güncelle")

val_image_paths = []
val_domain_labels = []
val_mask_names = []
val_mask_labels = []

skipped_no_domain = 0
skipped_no_dataset = 0
skipped_no_mask = 0
skipped_no_mapping = 0

# Detaylı istatistik için
found_datasets = defaultdict(int)
found_domains = defaultdict(int)

# 🔄 RANDOM SAMPLING: Tüm örneklerden random seç (daha dengeli dağılım)
# Önce tüm geçerli örnekleri bul, sonra random seç
print(f"\n   🔄 Örnekler taranıyor ve geçerli olanlar toplanıyor...")
print(f"   ⚠️  Bu işlem biraz zaman alabilir...")

# Tüm örnekleri random shuffle et (daha dengeli dataset dağılımı için)
all_indices = list(range(len(stage1_all_image_paths)))
random.seed(42)
random.shuffle(all_indices)

# 🔄 PROGRESS BAR: Tüm örnekleri tara, geçerli olanları topla
for idx in tqdm(all_indices, 
                desc=f"🔄 Scanning {len(all_indices):,} samples", 
                total=len(all_indices), 
                ncols=100, 
                leave=True, 
                unit="img"):
    
    # Hedef sayıya ulaştıysak dur
    if len(val_image_paths) >= target_count:
        break
    img_path = stage1_all_image_paths[idx]
    domain_label = stage1_all_labels[idx]
    
    # Domain name bul - label_to_domain'ın key tipi değişebilir
    domain_name = label_to_domain.get(domain_label) or label_to_domain.get(str(domain_label))
    if domain_name is None and isinstance(domain_label, (np.integer, np.int64)):
        domain_name = label_to_domain.get(int(domain_label))
    
    if domain_name is None:
        skipped_no_domain += 1
        continue
    
    # Image path'den dataset key'i çıkar
    # Path format: .../inpainting/{parent}/{model}/.../{filename}
    path_parts = img_path.replace('\\', '/').split('/')
    
    dataset_key = None
    for i, part in enumerate(path_parts):
        if part == 'inpainting' and i + 2 < len(path_parts):
            parent = path_parts[i + 1]
            model = path_parts[i + 2]
            dataset_key = f"{parent}_{model}"
            break
    
    if dataset_key is None or dataset_key not in mask_dict_by_dataset:
        skipped_no_dataset += 1
        continue
    
    found_datasets[dataset_key] += 1
    
    # Filename'den mask name bul
    filename = os.path.basename(img_path)
    filename_no_ext = os.path.splitext(filename)[0]
    
    mask_dict = mask_dict_by_dataset[dataset_key]
    mask_name = None
    
    # Key arama - Cell 6'daki mantıkla aynı
    for key in [filename, filename_no_ext, 
                filename_no_ext + '.png', filename_no_ext + '.jpg', filename_no_ext + '.jpeg']:
        if key in mask_dict:
            mask_name = mask_dict[key]
            break
    
    if mask_name is None:
        skipped_no_mask += 1
        continue
    
    # Mask label (domain-specific)
    mask_label = get_mask_label_for_domain(mask_name, domain_name, stage2_mappings)
    
    if mask_label is None:
        skipped_no_mapping += 1
        continue
    
    found_domains[domain_name] += 1
    
    # Tüm bilgileri ekle
    val_image_paths.append(img_path)
    val_domain_labels.append(domain_label)
    val_mask_names.append(mask_name)
    val_mask_labels.append(mask_label)

print(f"\n   ✅ {len(val_image_paths):,} geçerli örnek hazır")
print(f"   📊 Hedef: {target_count:,} | Bulunan: {len(val_image_paths):,} | Fark: {target_count - len(val_image_paths):+,}")

total_skipped = skipped_no_domain + skipped_no_dataset + skipped_no_mask + skipped_no_mapping
total_processed = len(val_image_paths) + total_skipped
if total_skipped > 0:
    print(f"\n   ⚠️ Atlanan örnek detayları (toplam {total_processed:,} örnek taranıldı):")
    print(f"      Domain bulunamadı: {skipped_no_domain:,} ({skipped_no_domain/total_processed*100:.1f}%)")
    print(f"      Dataset bulunamadı: {skipped_no_dataset:,} ({skipped_no_dataset/total_processed*100:.1f}%)")
    print(f"      Mask bulunamadı: {skipped_no_mask:,} ({skipped_no_mask/total_processed*100:.1f}%)")
    print(f"      Mask label bulunamadı: {skipped_no_mapping:,} ({skipped_no_mapping/total_processed*100:.1f}%)")
    print(f"      TOPLAM atlanan: {total_skipped:,} ({total_skipped/total_processed*100:.1f}%)")
    print(f"      ✅ Geçerli: {len(val_image_paths):,} ({len(val_image_paths)/total_processed*100:.1f}%)")

print(f"\n   📊 Bulunan dataset dağılımı:")
for ds, count in sorted(found_datasets.items(), key=lambda x: -x[1]):
    print(f"      {ds}: {count:,}")

print(f"\n   📊 Bulunan domain dağılımı:")
for dom, count in sorted(found_domains.items(), key=lambda x: -x[1]):
    print(f"      {dom}: {count:,}")

# =============================================
# 4️⃣ DATASET VE DATALOADER OLUŞTUR
# =============================================

print("\n4️⃣ Dataset ve DataLoader oluşturuluyor...")

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

hierarchical_val_dataset = HierarchicalValidationDataset(
    image_paths=val_image_paths,
    domain_labels=val_domain_labels,
    mask_names=val_mask_names,
    mask_labels=val_mask_labels,
    domain_names_list=DOMAIN_NAMES,
    transform=val_transform
)

# ✨ OPTIMIZED Hierarchical Validation DataLoader
hierarchical_val_loader = DataLoader(
    hierarchical_val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,  # ✨ Multi-threaded loading
    pin_memory=True,
    drop_last=False,
    persistent_workers=True if NUM_WORKERS > 0 else False,  # ✨ Worker'ları canlı tut
    prefetch_factor=2 if NUM_WORKERS > 0 else None  # ✨ Önden 2 batch yükle
)

print(f"   ✅ OPTIMIZED DataLoader hazır")
print(f"   Batch sayısı: {len(hierarchical_val_loader)}")
print(f"   Workers: {NUM_WORKERS}")
print(f"   Toplam örnek: {len(hierarchical_val_dataset):,}")

# =============================================
# 5️⃣ DOMAIN DAĞILIMI
# =============================================

print("\n5️⃣ Domain dağılımı:")

domain_dist = defaultdict(int)
mask_dist_per_domain = defaultdict(lambda: defaultdict(int))

for domain_label, mask_name in zip(val_domain_labels, val_mask_names):
    domain_name = label_to_domain.get(domain_label) or label_to_domain.get(str(domain_label))
    domain_dist[domain_name] += 1
    mask_dist_per_domain[domain_name][mask_name] += 1

print(f"\n   {'Domain':<15} {'Samples':>10} {'Unique Masks':>12} {'%':>8}")
print(f"   {'-'*50}")

total_samples = len(val_domain_labels)
for domain in DOMAIN_NAMES:
    count = domain_dist[domain]
    unique_masks = len(mask_dist_per_domain[domain])
    pct = (count / total_samples) * 100 if total_samples > 0 else 0
    print(f"   {domain:<15} {count:>10,} {unique_masks:>12} {pct:>7.1f}%")

print(f"   {'-'*50}")
print(f"   {'TOPLAM':<15} {total_samples:>10,}")

# =============================================
# 6️⃣ DOĞRULAMA TESTİ (OPTİMİZE - ATLANDI)
# =============================================

print("\n6️⃣ Doğrulama testi...")

# ⚡ OPTİMİZASYON: İlk batch testi atlanıyor (Cell 20'de test edilecek)
# Windows'ta multi-worker başlatma yavaş olabilir, direkt Cell 20'ye geç
print(f"   ⏭️  İlk batch testi atlanıyor (Cell 20'de test edilecek)")
print(f"   ✅ DataLoader hazır: {len(hierarchical_val_loader)} batch")
print(f"   ✅ Toplam örnek: {len(hierarchical_val_dataset):,}")
print(f"   💡 Gerçek test Cell 20'de yapılacak (model inference)")

print("\n" + "="*70)
print("✅ HİYERARŞİK VALİDASYON LOADER HAZIR!")
print("="*70)



In [ ]:
# =============================================
# ✨ YENİ: DİNAMİK ALPHA FONKSİYONU
# =============================================

def get_dynamic_alpha(domain_name, stage2_models):
    """
    ✨ YENİ: Domain'in Stage1 ve Stage2 accuracy'sine göre dinamik alpha belirle
    
    Args:
        domain_name: Domain adı (URBAN, INDOOR, vb.)
        stage2_models: Stage2 modelleri dict'i (val_acc bilgisi içerir)
    
    Returns:
        alpha: Stage1 ağırlığı (0-1 arası)
    """
    # Stage2 accuracy'yi al
    if domain_name in stage2_models:
        stage2_acc = stage2_models[domain_name].get('val_acc', 0.7)
    else:
        stage2_acc = 0.7  # Varsayılan
    
    # Domain'e göre strateji (gerçek accuracy sonuçlarına göre)
    if domain_name == 'INDOOR':
        # Stage1: 62.47%, Stage2: 87.32% → Stage2'ye çok güven
        return 0.2  # Stage1: %20, Stage2: %80
    
    elif domain_name == 'URBAN':
        # Stage1: 84.05%, Stage2: 63.76% → Stage1'e güven
        return 0.6  # Stage1: %60, Stage2: %40
    
    elif domain_name == 'CLOTHING':
        # Stage1: 7.27%, Stage2: 75.00% → Stage2'ye çok güven
        return 0.1  # Stage1: %10, Stage2: %90
    
    elif domain_name == 'HUMAN_BODY':
        # Stage1: 10.53%, Stage2: 0.00% → Her ikisi de kötü, dengeli ama Stage2'ye biraz daha güven
        return 0.4  # Stage1: %40, Stage2: %60
    
    elif domain_name == 'BACKGROUND':
        # Stage1: 100%, Stage2: 0% → Stage1'e güven
        return 0.7  # Stage1: %70, Stage2: %30
    
    else:
        # Varsayılan: Stage2 accuracy'sine göre
        if stage2_acc > 0.85:
            return 0.2  # Stage2 çok iyi
        elif stage2_acc > 0.70:
            return 0.4  # Stage2 iyi
        else:
            return 0.6  # Stage2 kötü → Stage1'e güven

print("✅ Dinamik alpha fonksiyonu tanımlandı")


In [ ]:
"""
🔄 HÜCRE 20 - HİYERARŞİK TEST PIPELINE (STAGE 1 → STAGE 2)
✨ YENİ: Multi-Expert Routing + Hybrid Scoring

URBAN/INDOOR/BACKGROUND karışıklığını çözmek için:
- Stage1 confidence düşükse VE karışan domain'lerdeyse
- Tüm karışan domain'lerin Stage2 modeline sor
- Hybrid Score = α × P(domain) + (1-α) × P(mask) × confidence_factor
- En yüksek hybrid score'u seç
"""

print("\n" + "="*70)
print("🔄 HİYERARŞİK TEST PIPELINE + MULTI-EXPERT ROUTING")
print("="*70)

# =============================================
# 0️⃣ HYBRID SCORING FONKSİYONU
# =============================================

def calculate_entropy(probs):
    """Probability dağılımının entropy'sini hesapla"""
    probs = np.clip(probs, 1e-10, 1.0)  # Log(0) önle
    entropy = -np.sum(probs * np.log(probs))
    return entropy

def calculate_hybrid_score(domain_prob, mask_probs, alpha=0.3):
    """
    Hybrid scoring: Stage1 + Stage2 + Entropy kombinasyonu
    
    Args:
        domain_prob: Stage1'den gelen bu domain'in olasılığı
        mask_probs: Stage2'den gelen mask olasılıkları (numpy array)
        alpha: Stage1 ağırlığı (0-1 arası)
    
    Returns:
        hybrid_score: Kombine skor
        mask_confidence: En yüksek mask olasılığı
        mask_pred: Tahmin edilen mask label
        confidence_factor: Entropy bazlı güven faktörü
    """
    # Mask prediction
    mask_confidence = np.max(mask_probs)
    mask_pred = np.argmax(mask_probs)
    
    # Entropy hesapla (düşük entropy = yüksek güven)
    entropy = calculate_entropy(mask_probs)
    max_entropy = np.log(len(mask_probs))  # Uniform dağılım entropy'si
    
    # Confidence factor: 0 (çok belirsiz) → 1 (çok emin)
    confidence_factor = 1 - (entropy / max_entropy) if max_entropy > 0 else 1.0
    
    # Hybrid score hesapla
    # α × P(domain) + (1-α) × P(mask) × confidence_factor
    hybrid_score = alpha * domain_prob + (1 - alpha) * mask_confidence * confidence_factor
    
    return hybrid_score, mask_confidence, mask_pred, confidence_factor

print("✅ Hybrid scoring fonksiyonu tanımlandı")

# =============================================
# 1️⃣ STAGE 1 MODEL YÜKLE
# =============================================

print("\n1️⃣ Stage 1 model yükleniyor...")

stage1_model_path = hierarchical_config['stage1']['model_path']
if not os.path.exists(stage1_model_path):
    stage1_model_path = os.path.join("eğitim_sonuçları", NOTEBOOK_NAME, "stage1_domain", "models", "best_model.pth")

stage1_checkpoint = torch.load(stage1_model_path, map_location=device, weights_only=False)
stage1_model = DomainClassifier(num_domains=NUM_DOMAINS).to(device)
stage1_model.load_state_dict(stage1_checkpoint['model_state_dict'])
stage1_model.eval()
# ✨ GPU optimizasyonu: Model'i GPU'da tut ve inference için hazırla
torch.cuda.empty_cache()  # Önceki cache'i temizle

print(f"   ✅ Stage 1 model yüklendi")
print(f"   Path: {stage1_model_path}")
print(f"   Val Accuracy: {stage1_checkpoint.get('val_acc', 0)*100:.2f}%")

# =============================================
# 2️⃣ STAGE 2 MODELS YÜKLE
# =============================================

print("\n2️⃣ Stage 2 models yükleniyor...")

stage2_models = {}

for domain_name in DOMAIN_NAMES:
    model_path = hierarchical_config['stage2'][domain_name]['model_path']
    if not os.path.exists(model_path):
        model_path = os.path.join("eğitim_sonuçları", NOTEBOOK_NAME, 
                                  f"stage2_{domain_name.lower()}", "models", "best_model.pth")
    
    if not os.path.exists(model_path):
        print(f"   ⚠️  {domain_name}: Model bulunamadı!")
        continue
    
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    num_masks = checkpoint['num_masks']
    
    model = DomainMaskDetector(num_masks=num_masks).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    # ✨ GPU optimizasyonu: Model'i GPU'da tut
    
    stage2_models[domain_name] = {
        'model': model,
        'num_masks': num_masks,
        'label_to_mask': checkpoint['label_to_mask'],
        'val_acc': checkpoint.get('val_acc', 0)
    }
    
    print(f"   ✅ {domain_name}: {num_masks} masks, Acc={checkpoint.get('val_acc', 0)*100:.2f}%")

print(f"\n   Toplam {len(stage2_models)} Stage 2 model yüklendi")
# ✨ GPU cache temizle ve hazır ol
torch.cuda.empty_cache()
torch.cuda.synchronize()  # GPU işlemlerinin tamamlanmasını bekle

# ✨ GPU kullanımını kontrol et
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    print(f"   ✅ Tüm modeller GPU'da hazır")
    print(f"   📊 GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")
else:
    print(f"   ⚠️  CUDA kullanılamıyor! CPU'da çalışıyor olabilir.")

# =============================================
# 3️⃣ HİYERARŞİK TEST BAŞLAT (MULTI-EXPERT ROUTING İLE)
# =============================================

print("\n3️⃣ Hierarchical test başlatılıyor...")
print(f"   Toplam örnek: {len(hierarchical_val_dataset):,}")
print(f"   Batch sayısı: {len(hierarchical_val_loader)}")
print(f"   🚀 GPU Optimizasyonları:")
print(f"      • Mixed Precision (AMP): {'✅ Aktif (FP16 - %40 VRAM tasarruf)' if USE_AMP else '❌ Kapalı'}")
print(f"      • Non-blocking Transfer: ✅ Aktif (CPU-GPU overlap)")
print(f"      • Device: {device}")
print(f"      • CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"      • GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"      • GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Multi-expert routing ayarlarını göster
print(f"\n   🔄 MULTI-EXPERT ROUTING:")
print(f"      Aktif: {'✅ Evet' if USE_MULTI_EXPERT_ROUTING else '❌ Hayır'}")
if USE_MULTI_EXPERT_ROUTING:
    print(f"      Karışan Domain'ler: {CONFUSING_DOMAINS}")
    print(f"      Routing Threshold: {ROUTING_THRESHOLD}")
    print(f"      Hybrid Alpha: {HYBRID_ALPHA}")

# Sonuçları saklamak için listeler
all_results = []

# İstatistikler
stats = {
    'normal_routing': 0,       # Normal routing kullanıldı
    'multi_expert_routing': 0,  # Multi-expert routing kullanıldı
    'multi_expert_changed': 0,  # Multi-expert domain'i değiştirdi
    'multi_expert_same': 0      # Multi-expert aynı domain'i seçti
}

# ✨ GPU optimizasyonu: Inference için hazır
torch.cuda.empty_cache()
torch.cuda.synchronize()

with torch.no_grad():
    # ✨ Mixed Precision context manager (tüm inference için)
    for batch_idx, (images, true_domains, true_masks, sample_indices) in enumerate(
        tqdm(hierarchical_val_loader, 
             desc="🔄 Hierarchical Testing + Multi-Expert",
             total=len(hierarchical_val_loader),
             ncols=100,
             leave=True,
             unit="batch",
             bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]")
    ):
        images = images.to(device, non_blocking=True)  # ✨ Non-blocking transfer
        batch_size = images.size(0)
        
        # ============================================
        # STAGE 1: Domain Classification (GPU OPTIMIZED)
        # ============================================
        
        # ✨ Mixed Precision (AMP) ile GPU'da işle
        with autocast(enabled=USE_AMP):
            domain_outputs = stage1_model(images)
        
        domain_probs_tensor = torch.softmax(domain_outputs, dim=1)
        domain_confidences, pred_domains = torch.max(domain_probs_tensor, 1)
        
        # CPU'ya taşı
        domain_probs_np = domain_probs_tensor.cpu().numpy()  # Tüm domain olasılıkları
        pred_domains = pred_domains.cpu().numpy()
        domain_confidences = domain_confidences.cpu().numpy()
        true_domains_np = true_domains.numpy()
        true_masks_np = true_masks.numpy()
        sample_indices_np = sample_indices.numpy()
        
        # ============================================
        # STAGE 2: Mask Classification
        # ✨ YENİ: Multi-Expert Routing ile
        # ============================================
        
        for i in range(batch_size):
            img = images[i:i+1]  # Keep batch dimension
            
            true_domain_label = true_domains_np[i]
            pred_domain_label = pred_domains[i]
            domain_conf = domain_confidences[i]
            true_mask_label = true_masks_np[i]
            sample_idx = sample_indices_np[i]
            all_domain_probs = domain_probs_np[i]  # Tüm domain olasılıkları
            
            # Domain names
            true_domain_name = label_to_domain.get(true_domain_label) or label_to_domain.get(str(true_domain_label))
            original_pred_domain_name = label_to_domain.get(pred_domain_label) or label_to_domain.get(str(pred_domain_label))
            
            # True mask name (dataset'ten)
            true_mask_name = hierarchical_val_dataset.mask_names[sample_idx]
            
            # ════════════════════════════════════════════════════════════════
            # 🔄 MULTI-EXPERT ROUTING KARARI
            # ════════════════════════════════════════════════════════════════
            
            use_multi_expert = False
            final_pred_domain_name = original_pred_domain_name
            routing_method = "normal"
            
            if USE_MULTI_EXPERT_ROUTING:
                # Koşul: Düşük confidence VE karışan domain'lerden biri
                if domain_conf < ROUTING_THRESHOLD and original_pred_domain_name in CONFUSING_DOMAINS:
                    use_multi_expert = True
            
            if use_multi_expert:
                # ════════════════════════════════════════════════════════════
                # 🎯 MULTI-EXPERT: Tüm karışan domain'lere sor
                # ════════════════════════════════════════════════════════════
                
                candidates = []
                
                for domain_name in CONFUSING_DOMAINS:
                    if domain_name not in stage2_models:
                        continue
                    
                    # Bu domain için Stage2 çıktısı al (GPU'da Mixed Precision ile)
                    s2_model = stage2_models[domain_name]['model']
                    with autocast(enabled=USE_AMP):
                        s2_outputs = s2_model(img)
                    s2_probs = torch.softmax(s2_outputs, dim=1).cpu().numpy()[0]
                    
                    # Bu domain'in Stage1 olasılığını bul
                    domain_idx = domain_to_label[domain_name]
                    domain_prob = all_domain_probs[domain_idx]
                    
                    # Hybrid score hesapla (dinamik alpha ile)
                    if USE_DYNAMIC_ALPHA:
                        dynamic_alpha = get_dynamic_alpha(domain_name, stage2_models)
                        hybrid_score, mask_conf, mask_pred, conf_factor = calculate_hybrid_score(
                            domain_prob, s2_probs, alpha=dynamic_alpha
                        )
                    else:
                        hybrid_score, mask_conf, mask_pred, conf_factor = calculate_hybrid_score(
                            domain_prob, s2_probs, alpha=HYBRID_ALPHA
                        )
                    
                    # Label to mask name
                    l2m = stage2_models[domain_name]['label_to_mask']
                    mask_name = l2m.get(int(mask_pred)) or l2m.get(str(mask_pred), "UNKNOWN")
                    
                    candidates.append({
                        'domain': domain_name,
                        'domain_prob': domain_prob,
                        'mask_pred': int(mask_pred),
                        'mask_name': mask_name,
                        'mask_confidence': mask_conf,
                        'confidence_factor': conf_factor,
                        'hybrid_score': hybrid_score
                    })
                
                # En yüksek hybrid score'u seç
                if candidates:
                    best = max(candidates, key=lambda x: x['hybrid_score'])
                    
                    final_pred_domain_name = best['domain']
                    pred_mask_label = best['mask_pred']
                    pred_mask_name = best['mask_name']
                    mask_confidence = best['mask_confidence']
                    routing_method = "multi_expert"
                    
                    stats['multi_expert_routing'] += 1
                    
                    if final_pred_domain_name != original_pred_domain_name:
                        stats['multi_expert_changed'] += 1
                    else:
                        stats['multi_expert_same'] += 1
                else:
                    # Fallback: Normal routing
                    use_multi_expert = False
            
            if not use_multi_expert:
                # ════════════════════════════════════════════════════════════
                # 📍 NORMAL ROUTING: Sadece tahmin edilen domain'e sor
                # ════════════════════════════════════════════════════════════
                
                pred_mask_label = -1
                pred_mask_name = "UNKNOWN"
                mask_confidence = 0.0
                
                if original_pred_domain_name in stage2_models:
                    s2_model = stage2_models[original_pred_domain_name]['model']
                    # ✨ Mixed Precision (AMP) ile GPU'da işle
                    with autocast(enabled=USE_AMP):
                        s2_outputs = s2_model(img)
                    s2_probs = torch.softmax(s2_outputs, dim=1)
                    mask_conf, mask_pred = torch.max(s2_probs, 1)
                    
                    pred_mask_label = mask_pred.item()
                    mask_confidence = mask_conf.item()
                    
                    l2m = stage2_models[original_pred_domain_name]['label_to_mask']
                    pred_mask_name = l2m.get(pred_mask_label) or l2m.get(str(pred_mask_label), "UNKNOWN")
                
                final_pred_domain_name = original_pred_domain_name
                routing_method = "normal"
                stats['normal_routing'] += 1
            
            # ════════════════════════════════════════════════════════════════
            # 📊 SONUÇLARI DEĞERLENDIR
            # ════════════════════════════════════════════════════════════════
            
            # Final domain label'ı bul
            final_pred_domain_label = domain_to_label.get(final_pred_domain_name, pred_domain_label)
            
            # Domain doğruluğu (multi-expert sonrası)
            domain_correct = (final_pred_domain_label == true_domain_label)
            
            # Mask doğruluğu
            if domain_correct:
                expected_mask_label = get_mask_label_for_domain(true_mask_name, true_domain_name, stage2_mappings)
                mask_correct = (pred_mask_label == expected_mask_label) if expected_mask_label is not None else False
            else:
                mask_correct = False
            
            # Sonucu kaydet
            result = {
                'sample_idx': sample_idx,
                'image_path': hierarchical_val_dataset.image_paths[sample_idx],
                'true_domain_label': true_domain_label,
                'original_pred_domain_label': pred_domain_label,  # Stage1'in orijinal tahmini
                'final_pred_domain_label': final_pred_domain_label,  # Multi-expert sonrası
                'true_domain_name': true_domain_name,
                'original_pred_domain_name': original_pred_domain_name,
                'final_pred_domain_name': final_pred_domain_name,
                'domain_confidence': domain_conf,
                'domain_correct': domain_correct,
                'true_mask_label': true_mask_label,
                'pred_mask_label': pred_mask_label,
                'true_mask_name': true_mask_name,
                'pred_mask_name': pred_mask_name,
                'mask_confidence': mask_confidence,
                'mask_correct': mask_correct,
                'pipeline_correct': domain_correct and mask_correct,
                'routing_method': routing_method  # "normal" veya "multi_expert"
            }
            
            all_results.append(result)

# DataFrame oluştur
df_results = pd.DataFrame(all_results)

print(f"\n   ✅ Test tamamlandı: {len(df_results):,} örnek")

# =============================================
# 4️⃣ MULTI-EXPERT ROUTING İSTATİSTİKLERİ
# =============================================

print("\n4️⃣ Multi-Expert Routing İstatistikleri:")
print(f"   {'-'*50}")
print(f"   Normal Routing kullanıldı:        {stats['normal_routing']:>8,}")
print(f"   Multi-Expert Routing kullanıldı:  {stats['multi_expert_routing']:>8,}")
if stats['multi_expert_routing'] > 0:
    print(f"      → Domain DEĞİŞTİ:              {stats['multi_expert_changed']:>8,}")
    print(f"      → Domain AYNI kaldı:           {stats['multi_expert_same']:>8,}")
    change_rate = stats['multi_expert_changed'] / stats['multi_expert_routing'] * 100
    print(f"      → Değiştirme oranı:            {change_rate:>7.1f}%")

# =============================================
# 5️⃣ TEMEL METRİKLER
# =============================================

print("\n5️⃣ Temel metrikler hesaplanıyor...")

total_samples = len(df_results)
domain_correct_count = df_results['domain_correct'].sum()
domain_accuracy = domain_correct_count / total_samples

# Mask accuracy (sadece domain doğru olanlar için)
df_domain_correct = df_results[df_results['domain_correct'] == True]
mask_correct_count = df_domain_correct['mask_correct'].sum()
mask_accuracy_on_correct = mask_correct_count / len(df_domain_correct) if len(df_domain_correct) > 0 else 0

# Full pipeline accuracy
pipeline_correct_count = df_results['pipeline_correct'].sum()
pipeline_accuracy = pipeline_correct_count / total_samples

print(f"\n   📊 DOMAIN CLASSIFICATION (Multi-Expert sonrası):")
print(f"      Accuracy: {domain_accuracy*100:.2f}%")
print(f"      Doğru: {domain_correct_count:,} / {total_samples:,}")
print(f"      Yanlış: {total_samples - domain_correct_count:,}")

print(f"\n   📊 MASK DETECTION (Domain doğru olanlar için):")
print(f"      Accuracy: {mask_accuracy_on_correct*100:.2f}%")
print(f"      Doğru: {mask_correct_count:,} / {len(df_domain_correct):,}")

print(f"\n   📊 FULL PIPELINE (Domain + Mask):")
print(f"      Accuracy: {pipeline_accuracy*100:.2f}%")
print(f"      Doğru: {pipeline_correct_count:,} / {total_samples:,}")

# =============================================
# 6️⃣ MULTI-EXPERT ETKİSİ ANALİZİ
# =============================================

if USE_MULTI_EXPERT_ROUTING and stats['multi_expert_routing'] > 0:
    print("\n6️⃣ Multi-Expert Etkisi Analizi:")
    
    # Multi-expert kullanılan örnekleri filtrele
    df_multi_expert = df_results[df_results['routing_method'] == 'multi_expert']
    df_normal = df_results[df_results['routing_method'] == 'normal']
    
    # Multi-expert'in düzelttiği örnekler (orijinal yanlış, final doğru)
    df_fixed_by_multi = df_multi_expert[
        (df_multi_expert['original_pred_domain_label'] != df_multi_expert['true_domain_label']) &
        (df_multi_expert['final_pred_domain_label'] == df_multi_expert['true_domain_label'])
    ]
    
    # Multi-expert'in bozduğu örnekler (orijinal doğru, final yanlış)
    df_broken_by_multi = df_multi_expert[
        (df_multi_expert['original_pred_domain_label'] == df_multi_expert['true_domain_label']) &
        (df_multi_expert['final_pred_domain_label'] != df_multi_expert['true_domain_label'])
    ]
    
    print(f"   {'-'*50}")
    print(f"   Multi-Expert kullanılan toplam: {len(df_multi_expert):,}")
    print(f"   ✅ Multi-Expert'in DÜZELTTİĞİ:  {len(df_fixed_by_multi):,}")
    print(f"   ❌ Multi-Expert'in BOZDUĞU:     {len(df_broken_by_multi):,}")
    net_improvement = len(df_fixed_by_multi) - len(df_broken_by_multi)
    print(f"   📈 NET İYİLEŞTİRME:             {net_improvement:+,}")
    
    # Eğer net iyileştirme varsa, yüzde hesapla
    if net_improvement != 0:
        improvement_pct = net_improvement / total_samples * 100
        print(f"   📈 Pipeline accuracy artışı:   {improvement_pct:+.2f}%")
    
    # Orijinal vs Final karşılaştırma
    original_domain_correct = (df_results['original_pred_domain_label'] == df_results['true_domain_label']).sum()
    final_domain_correct = (df_results['final_pred_domain_label'] == df_results['true_domain_label']).sum()
    
    print(f"\n   📊 KARŞILAŞTIRMA:")
    print(f"      Orijinal Stage1 doğru:  {original_domain_correct:,} ({original_domain_correct/total_samples*100:.2f}%)")
    print(f"      Multi-Expert sonrası:   {final_domain_correct:,} ({final_domain_correct/total_samples*100:.2f}%)")
    print(f"      Fark:                   {final_domain_correct - original_domain_correct:+,}")

# Summary dict oluştur
test_summary = {
    'total_samples': total_samples,
    'domain_accuracy': domain_accuracy,
    'domain_correct': int(domain_correct_count),
    'domain_wrong': int(total_samples - domain_correct_count),
    'mask_accuracy_on_correct_domain': mask_accuracy_on_correct,
    'mask_correct': int(mask_correct_count),
    'pipeline_accuracy': pipeline_accuracy,
    'pipeline_correct': int(pipeline_correct_count),
    'multi_expert_stats': stats,
    'routing_threshold': ROUTING_THRESHOLD if USE_MULTI_EXPERT_ROUTING else None,
    'hybrid_alpha': HYBRID_ALPHA if USE_MULTI_EXPERT_ROUTING else None
}

print("\n" + "="*70)
print("✅ HİYERARŞİK TEST + MULTI-EXPERT ROUTING TAMAMLANDI!")
print("="*70)


In [ ]:
"""
🔍 HÜCRE 21 - DETAYLI HATA ANALİZİ
Hata tiplerini kategorize et ve detaylı analiz yap
"""

print("\n" + "="*70)
print("🔍 DETAYLI HATA ANALİZİ")
print("="*70)

# =============================================
# 1️⃣ HATA KATEGORİLERİ
# =============================================

print("\n1️⃣ Hata kategorileri oluşturuluyor...")

# Kategorilere ayır
df_stage1_correct_stage2_correct = df_results[(df_results['domain_correct'] == True) & (df_results['mask_correct'] == True)]
df_stage1_correct_stage2_wrong = df_results[(df_results['domain_correct'] == True) & (df_results['mask_correct'] == False)]
df_stage1_wrong = df_results[df_results['domain_correct'] == False]

error_categories = {
    'Stage1✓_Stage2✓': len(df_stage1_correct_stage2_correct),
    'Stage1✓_Stage2✗': len(df_stage1_correct_stage2_wrong),
    'Stage1✗': len(df_stage1_wrong)
}

print(f"\n   📊 HATA KATEGORİLERİ:")
print(f"   {'-'*60}")
print(f"   {'Kategori':<30} {'Sayı':>10} {'Oran':>10}")
print(f"   {'-'*60}")

for category, count in error_categories.items():
    pct = (count / total_samples) * 100
    print(f"   {category:<30} {count:>10,} {pct:>9.2f}%")

print(f"   {'-'*60}")
print(f"   {'TOPLAM':<30} {total_samples:>10,}")

# =============================================
# 2️⃣ DOMAIN BAZINDA HATA ANALİZİ
# =============================================

print("\n2️⃣ Domain bazında hata analizi...")

domain_error_analysis = []

for domain in DOMAIN_NAMES:
    df_domain = df_results[df_results['true_domain_name'] == domain]
    
    if len(df_domain) == 0:
        continue
    
    domain_correct = df_domain['domain_correct'].sum()
    domain_wrong = len(df_domain) - domain_correct
    
    df_domain_correct_subset = df_domain[df_domain['domain_correct'] == True]
    mask_correct = df_domain_correct_subset['mask_correct'].sum() if len(df_domain_correct_subset) > 0 else 0
    mask_wrong = len(df_domain_correct_subset) - mask_correct if len(df_domain_correct_subset) > 0 else 0
    
    pipeline_correct = df_domain['pipeline_correct'].sum()
    
    domain_error_analysis.append({
        'domain': domain,
        'total': len(df_domain),
        'stage1_correct': domain_correct,
        'stage1_wrong': domain_wrong,
        'stage1_accuracy': domain_correct / len(df_domain),
        'stage2_correct': mask_correct,
        'stage2_wrong': mask_wrong,
        'stage2_accuracy': mask_correct / domain_correct if domain_correct > 0 else 0,
        'pipeline_correct': pipeline_correct,
        'pipeline_accuracy': pipeline_correct / len(df_domain)
    })

df_domain_analysis = pd.DataFrame(domain_error_analysis)

print(f"\n   {'Domain':<15} {'Total':>8} {'S1 Acc':>10} {'S2 Acc':>10} {'Pipeline':>10}")
print(f"   {'-'*60}")

for _, row in df_domain_analysis.iterrows():
    print(f"   {row['domain']:<15} {row['total']:>8,} "
          f"{row['stage1_accuracy']*100:>9.2f}% "
          f"{row['stage2_accuracy']*100:>9.2f}% "
          f"{row['pipeline_accuracy']*100:>9.2f}%")

# =============================================
# 3️⃣ EN ÇOK KARIŞTIRILAN DOMAIN ÇİFTLERİ
# =============================================

print("\n3️⃣ En çok karıştırılan domain çiftleri (Multi-Expert sonrası)...")

domain_confusion_pairs = df_stage1_wrong.groupby(['true_domain_name', 'final_pred_domain_name']).size().reset_index(name='count')
domain_confusion_pairs = domain_confusion_pairs.sort_values('count', ascending=False)

print(f"\n   {'True Domain':<15} {'Pred Domain':<15} {'Count':>8} {'%':>8}")
print(f"   {'-'*50}")

for _, row in domain_confusion_pairs.head(10).iterrows():
    pct = (row['count'] / len(df_stage1_wrong)) * 100 if len(df_stage1_wrong) > 0 else 0
    print(f"   {row['true_domain_name']:<15} {row['final_pred_domain_name']:<15} {row['count']:>8,} {pct:>7.2f}%")

# =============================================
# 4️⃣ EN ÇOK KARIŞTIRILAN MASK'LAR (Domain Doğru Olanlar İçin)
# =============================================

print("\n4️⃣ En çok karıştırılan mask'lar (domain doğru olanlar için)...")

for domain in DOMAIN_NAMES:
    df_domain_s2_wrong = df_stage1_correct_stage2_wrong[df_stage1_correct_stage2_wrong['true_domain_name'] == domain]
    
    if len(df_domain_s2_wrong) == 0:
        continue
    
    mask_confusion = df_domain_s2_wrong.groupby(['true_mask_name', 'pred_mask_name']).size().reset_index(name='count')
    mask_confusion = mask_confusion.sort_values('count', ascending=False)
    
    print(f"\n   🎯 {domain}:")
    for _, row in mask_confusion.head(5).iterrows():
        print(f"      {row['true_mask_name']:<20} → {row['pred_mask_name']:<20} ({row['count']})")

# =============================================
# 5️⃣ CONFIDENCE ANALİZİ
# =============================================

print("\n5️⃣ Confidence analizi...")

# Domain confidence by correctness
avg_conf_correct = df_results[df_results['domain_correct'] == True]['domain_confidence'].mean()
avg_conf_wrong = df_results[df_results['domain_correct'] == False]['domain_confidence'].mean()

print(f"\n   📊 Domain Confidence:")
print(f"      Doğru tahminler ortalama confidence: {avg_conf_correct:.4f}")
print(f"      Yanlış tahminler ortalama confidence: {avg_conf_wrong:.4f}")
print(f"      Fark: {avg_conf_correct - avg_conf_wrong:.4f}")

# Low confidence analysis
low_conf_threshold = 0.7
df_low_conf = df_results[df_results['domain_confidence'] < low_conf_threshold]
low_conf_wrong = df_low_conf[df_low_conf['domain_correct'] == False]

print(f"\n   📊 Düşük Confidence (<{low_conf_threshold}) Analizi:")
print(f"      Toplam düşük confidence: {len(df_low_conf):,} ({len(df_low_conf)/total_samples*100:.2f}%)")
print(f"      Bunların yanlış olanları: {len(low_conf_wrong):,} ({len(low_conf_wrong)/len(df_low_conf)*100:.2f}% of low conf)")

# High confidence analysis
high_conf_threshold = 0.9
df_high_conf = df_results[df_results['domain_confidence'] >= high_conf_threshold]
high_conf_correct = df_high_conf[df_high_conf['domain_correct'] == True]

print(f"\n   📊 Yüksek Confidence (≥{high_conf_threshold}) Analizi:")
print(f"      Toplam yüksek confidence: {len(df_high_conf):,} ({len(df_high_conf)/total_samples*100:.2f}%)")
print(f"      Bunların doğru olanları: {len(high_conf_correct):,} ({len(high_conf_correct)/len(df_high_conf)*100:.2f}% of high conf)")

# =============================================
# 6️⃣ HATA ANALİZİ ÖZET
# =============================================

error_analysis_summary = {
    'error_categories': error_categories,
    'domain_analysis': df_domain_analysis.to_dict('records'),
    'top_domain_confusions': domain_confusion_pairs.head(10).to_dict('records'),
    'avg_confidence_correct': avg_conf_correct,
    'avg_confidence_wrong': avg_conf_wrong,
    'low_confidence_samples': len(df_low_conf),
    'low_confidence_error_rate': len(low_conf_wrong) / len(df_low_conf) if len(df_low_conf) > 0 else 0,
    'high_confidence_samples': len(df_high_conf),
    'high_confidence_accuracy': len(high_conf_correct) / len(df_high_conf) if len(df_high_conf) > 0 else 0
}

print("\n" + "="*70)
print("✅ HATA ANALİZİ TAMAMLANDI!")
print("="*70)


In [ ]:
"""
📊 HÜCRE 22 - PROFESYONEL GÖRSELLEŞTİRME - BÖLÜM 1
Confusion Matrices ve Performance Charts
"""

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap

print("\n" + "="*70)
print("📊 PROFESYONEL GÖRSELLEŞTİRME - BÖLÜM 1")
print("="*70)

# Çıktı klasörü
output_folder = "hierarchical_validation_test_results"
viz_folder = os.path.join(output_folder, "visualizations")
os.makedirs(viz_folder, exist_ok=True)

# Custom color palette
COLORS = {
    'correct': '#2ecc71',      # Green
    'wrong': '#e74c3c',        # Red
    'domain_correct': '#3498db',  # Blue
    'mask_wrong': '#f39c12',   # Orange
    'neutral': '#95a5a6',      # Gray
    'background': '#2c3e50',   # Dark blue
    'accent': '#9b59b6'        # Purple
}

# Profesyonel style ayarları
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = '#f8f9fa'
plt.rcParams['grid.alpha'] = 0.3

# =============================================
# 1️⃣ STAGE 1 CONFUSION MATRIX (5x5 Domain) - Multi-Expert Sonrası
# =============================================

print("\n1️⃣ Stage 1 Confusion Matrix oluşturuluyor (Multi-Expert sonrası)...")

# Confusion matrix hesapla - final (multi-expert sonrası) tahminleri kullan
true_domains = df_results['true_domain_label'].values
pred_domains = df_results['final_pred_domain_label'].values

cm_domain = confusion_matrix(true_domains, pred_domains, labels=range(NUM_DOMAINS))
cm_domain_normalized = cm_domain.astype('float') / cm_domain.sum(axis=1)[:, np.newaxis] * 100

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Custom colormap
cmap_counts = LinearSegmentedColormap.from_list('custom_blues', ['#ffffff', '#3498db', '#1a5276'])
cmap_pct = LinearSegmentedColormap.from_list('custom_greens', ['#ffffff', '#27ae60', '#145a32'])

# Sol: Raw counts
sns.heatmap(cm_domain, annot=True, fmt='d', cmap=cmap_counts,
            xticklabels=DOMAIN_NAMES, yticklabels=DOMAIN_NAMES,
            ax=axes[0], cbar_kws={'label': 'Count', 'shrink': 0.8},
            annot_kws={'size': 12, 'weight': 'bold'},
            linewidths=0.5, linecolor='white')
axes[0].set_xlabel('Predicted Domain', fontsize=13, fontweight='bold', labelpad=10)
axes[0].set_ylabel('True Domain', fontsize=13, fontweight='bold', labelpad=10)
axes[0].set_title('Domain Classification (Multi-Expert)\n(Absolute Counts)', fontsize=15, fontweight='bold', pad=15)
axes[0].tick_params(axis='x', rotation=45, labelsize=11)
axes[0].tick_params(axis='y', rotation=0, labelsize=11)

# Sağ: Percentages
sns.heatmap(cm_domain_normalized, annot=True, fmt='.1f', cmap=cmap_pct,
            xticklabels=DOMAIN_NAMES, yticklabels=DOMAIN_NAMES,
            ax=axes[1], cbar_kws={'label': 'Percentage (%)', 'shrink': 0.8},
            annot_kws={'size': 12, 'weight': 'bold'},
            linewidths=0.5, linecolor='white', vmin=0, vmax=100)
axes[1].set_xlabel('Predicted Domain', fontsize=13, fontweight='bold', labelpad=10)
axes[1].set_ylabel('True Domain', fontsize=13, fontweight='bold', labelpad=10)
axes[1].set_title('Domain Classification (Multi-Expert)\n(Row-Normalized %)', fontsize=15, fontweight='bold', pad=15)
axes[1].tick_params(axis='x', rotation=45, labelsize=11)
axes[1].tick_params(axis='y', rotation=0, labelsize=11)

# Accuracy text
overall_acc = np.trace(cm_domain) / cm_domain.sum()
fig.suptitle(f'Domain Classification Accuracy (Multi-Expert Routing): {overall_acc*100:.2f}%', 
             fontsize=16, fontweight='bold', y=1.02, color=COLORS['background'])

plt.tight_layout()
plt.savefig(os.path.join(viz_folder, 'stage1_domain_confusion_matrix.png'), 
            dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()

print(f"   ✅ Stage 1 confusion matrix kaydedildi")

# =============================================
# 2️⃣ STAGE 2 CONFUSION MATRICES (Her domain için)
# =============================================

print("\n2️⃣ Stage 2 Confusion Matrices oluşturuluyor...")

# Her domain için ayrı confusion matrix
fig, axes = plt.subplots(2, 3, figsize=(20, 14))
axes = axes.flatten()

for idx, domain in enumerate(DOMAIN_NAMES):
    ax = axes[idx]
    
    # Bu domain için doğru tahmin edilen örnekleri al
    df_domain_correct = df_results[(df_results['true_domain_name'] == domain) & 
                                    (df_results['domain_correct'] == True)]
    
    if len(df_domain_correct) == 0:
        ax.text(0.5, 0.5, f'{domain}\nNo data', ha='center', va='center', fontsize=14)
        ax.set_title(f'{domain} - No Data')
        continue
    
    # Mask labels (true and predicted)
    true_masks = df_domain_correct['true_mask_label'].values
    pred_masks = df_domain_correct['pred_mask_label'].values
    
    # Unique labels
    unique_labels = sorted(set(true_masks) | set(pred_masks))
    
    if len(unique_labels) == 0:
        ax.text(0.5, 0.5, f'{domain}\nNo valid masks', ha='center', va='center', fontsize=14)
        continue
    
    # Mask names for this domain
    l2m = stage2_models[domain]['label_to_mask'] if domain in stage2_models else {}
    mask_names = [l2m.get(l) or l2m.get(str(l), f"Mask{l}") for l in unique_labels]
    
    # Confusion matrix
    cm_mask = confusion_matrix(true_masks, pred_masks, labels=unique_labels)
    cm_mask_norm = cm_mask.astype('float') / (cm_mask.sum(axis=1)[:, np.newaxis] + 1e-10) * 100
    
    # Plot
    sns.heatmap(cm_mask_norm, annot=True, fmt='.0f', cmap='YlOrRd',
                xticklabels=mask_names, yticklabels=mask_names,
                ax=ax, cbar_kws={'shrink': 0.6},
                annot_kws={'size': max(6, 10 - len(unique_labels)//3)},
                linewidths=0.5, linecolor='white', vmin=0, vmax=100)
    
    # Accuracy for this domain
    domain_mask_acc = np.trace(cm_mask) / cm_mask.sum() if cm_mask.sum() > 0 else 0
    
    ax.set_xlabel('Predicted Mask', fontsize=10, fontweight='bold')
    ax.set_ylabel('True Mask', fontsize=10, fontweight='bold')
    ax.set_title(f'{domain}\nMask Accuracy: {domain_mask_acc*100:.1f}%\n({len(df_domain_correct):,} samples)', 
                fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=90, labelsize=max(6, 9 - len(unique_labels)//3))
    ax.tick_params(axis='y', rotation=0, labelsize=max(6, 9 - len(unique_labels)//3))

# Son subplot'u gizle
axes[5].axis('off')

# Başlık
fig.suptitle('Stage 2: Mask Detection Confusion Matrices by Domain\n(Only Domain-Correct Samples)', 
            fontsize=16, fontweight='bold', y=1.02)

plt.tight_layout()
plt.savefig(os.path.join(viz_folder, 'stage2_mask_confusion_matrices.png'), 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"   ✅ Stage 2 confusion matrices kaydedildi")

# =============================================
# 3️⃣ PER-DOMAIN ACCURACY BAR CHART
# =============================================

print("\n3️⃣ Per-domain accuracy chart oluşturuluyor...")

fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(DOMAIN_NAMES))
width = 0.28

# Accuracy values from domain analysis
s1_accs = [df_domain_analysis[df_domain_analysis['domain'] == d]['stage1_accuracy'].values[0] 
           if d in df_domain_analysis['domain'].values else 0 for d in DOMAIN_NAMES]
s2_accs = [df_domain_analysis[df_domain_analysis['domain'] == d]['stage2_accuracy'].values[0]
           if d in df_domain_analysis['domain'].values else 0 for d in DOMAIN_NAMES]
pipeline_accs = [df_domain_analysis[df_domain_analysis['domain'] == d]['pipeline_accuracy'].values[0]
                 if d in df_domain_analysis['domain'].values else 0 for d in DOMAIN_NAMES]

# Bars
bars1 = ax.bar(x - width, s1_accs, width, label='Stage 1 (Domain)', color=COLORS['domain_correct'], 
               edgecolor='white', linewidth=2, alpha=0.85)
bars2 = ax.bar(x, s2_accs, width, label='Stage 2 (Mask)', color=COLORS['correct'], 
               edgecolor='white', linewidth=2, alpha=0.85)
bars3 = ax.bar(x + width, pipeline_accs, width, label='Full Pipeline', color=COLORS['accent'], 
               edgecolor='white', linewidth=2, alpha=0.85)

# Target line
ax.axhline(y=0.85, color=COLORS['wrong'], linestyle='--', linewidth=2, 
          alpha=0.7, label='Target (85%)')

# Labels
ax.set_xlabel('Domain', fontsize=14, fontweight='bold', labelpad=10)
ax.set_ylabel('Accuracy', fontsize=14, fontweight='bold', labelpad=10)
ax.set_title('Hierarchical Pipeline Performance by Domain', fontsize=16, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels(DOMAIN_NAMES, fontsize=12, fontweight='bold')
ax.legend(fontsize=11, loc='upper right', framealpha=0.9)
ax.set_ylim([0, 1.05])
ax.grid(True, alpha=0.3, axis='y')

# Bar labels
for bars, values in [(bars1, s1_accs), (bars2, s2_accs), (bars3, pipeline_accs)]:
    for bar, val in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{val*100:.1f}%',
                ha='center', va='bottom', fontsize=9, fontweight='bold', rotation=0)

# Overall stats annotation
textstr = f'Overall Stage 1: {domain_accuracy*100:.1f}%\nOverall Stage 2: {mask_accuracy_on_correct*100:.1f}%\nOverall Pipeline: {pipeline_accuracy*100:.1f}%'
props = dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.9, edgecolor=COLORS['background'])
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(viz_folder, 'per_domain_accuracy.png'), 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"   ✅ Per-domain accuracy chart kaydedildi")

print("\n" + "="*70)
print("✅ GÖRSELLEŞTİRME BÖLÜM 1 TAMAMLANDI!")
print("="*70)


In [ ]:
"""
📊 HÜCRE 23 - PROFESYONEL GÖRSELLEŞTİRME - BÖLÜM 2
Error Flow Diagram, Confidence Distributions, Error Breakdown
"""

print("\n" + "="*70)
print("📊 PROFESYONEL GÖRSELLEŞTİRME - BÖLÜM 2")
print("="*70)

# =============================================
# 4️⃣ ERROR FLOW DIAGRAM (Sankey-style)
# =============================================

print("\n4️⃣ Error Flow Diagram oluşturuluyor...")

fig, ax = plt.subplots(figsize=(16, 10))

# Hesaplamalar
n_total = len(df_results)
n_s1_correct = len(df_stage1_correct_stage2_correct) + len(df_stage1_correct_stage2_wrong)
n_s1_wrong = len(df_stage1_wrong)
n_s2_correct = len(df_stage1_correct_stage2_correct)
n_s2_wrong = len(df_stage1_correct_stage2_wrong)

# Stage konumları
stages = ['Input\nSamples', 'Stage 1\nDomain', 'Stage 2\nMask', 'Output']
stage_x = [0, 1, 2, 3]

# Y pozisyonları (akış yükseklikleri)
y_positions = {
    'total': 0.5,
    's1_correct': 0.65,
    's1_wrong': 0.25,
    's2_correct': 0.75,
    's2_wrong': 0.55,
    'final_correct': 0.75,
    'final_wrong': 0.35
}

# Box'ları çiz
def draw_box(ax, x, y, width, height, color, label, count, pct):
    rect = plt.Rectangle((x - width/2, y - height/2), width, height, 
                         facecolor=color, edgecolor='white', linewidth=2, alpha=0.85)
    ax.add_patch(rect)
    ax.text(x, y, f'{label}\n{count:,}\n({pct:.1f}%)', 
            ha='center', va='center', fontsize=10, fontweight='bold', color='white')

# Akış çizgileri
def draw_flow(ax, x1, y1, x2, y2, width, color, alpha=0.4):
    from matplotlib.patches import FancyBboxPatch, ConnectionPatch
    # Basit çizgi çiz
    ax.fill([x1, x2, x2, x1], [y1-width/2, y2-width/2, y2+width/2, y1+width/2], 
            color=color, alpha=alpha, linewidth=0)

# Total input
draw_box(ax, 0, 0.5, 0.3, 0.35, COLORS['neutral'], 'Total Input', n_total, 100)

# Stage 1 outputs
s1_correct_pct = n_s1_correct/n_total*100
s1_wrong_pct = n_s1_wrong/n_total*100
draw_box(ax, 1, 0.7, 0.3, 0.25, COLORS['domain_correct'], 'Domain\nCorrect', n_s1_correct, s1_correct_pct)
draw_box(ax, 1, 0.25, 0.3, 0.2, COLORS['wrong'], 'Domain\nWrong', n_s1_wrong, s1_wrong_pct)

# Stage 2 outputs (sadece S1 doğru olanlar için)
s2_correct_pct = n_s2_correct/n_total*100
s2_wrong_pct = n_s2_wrong/n_total*100
draw_box(ax, 2, 0.8, 0.3, 0.2, COLORS['correct'], 'Mask\nCorrect', n_s2_correct, s2_correct_pct)
draw_box(ax, 2, 0.55, 0.3, 0.18, COLORS['mask_wrong'], 'Mask\nWrong', n_s2_wrong, s2_wrong_pct)

# Final outputs
final_correct = n_s2_correct
final_wrong = n_s1_wrong + n_s2_wrong
final_correct_pct = final_correct/n_total*100
final_wrong_pct = final_wrong/n_total*100
draw_box(ax, 3, 0.8, 0.3, 0.2, COLORS['correct'], 'SUCCESS', final_correct, final_correct_pct)
draw_box(ax, 3, 0.3, 0.3, 0.3, COLORS['wrong'], 'FAILED', final_wrong, final_wrong_pct)

# Akış çizgileri
# Input → S1 Correct
draw_flow(ax, 0.15, 0.6, 0.85, 0.7, 0.15, COLORS['domain_correct'])
# Input → S1 Wrong
draw_flow(ax, 0.15, 0.4, 0.85, 0.25, 0.1, COLORS['wrong'])
# S1 Correct → S2 Correct
draw_flow(ax, 1.15, 0.75, 1.85, 0.8, 0.1, COLORS['correct'])
# S1 Correct → S2 Wrong
draw_flow(ax, 1.15, 0.65, 1.85, 0.55, 0.08, COLORS['mask_wrong'])
# S1 Wrong → Final Wrong
draw_flow(ax, 1.15, 0.25, 2.85, 0.3, 0.08, COLORS['wrong'], alpha=0.3)
# S2 Correct → Final Correct
draw_flow(ax, 2.15, 0.8, 2.85, 0.8, 0.1, COLORS['correct'])
# S2 Wrong → Final Wrong
draw_flow(ax, 2.15, 0.55, 2.85, 0.35, 0.08, COLORS['mask_wrong'], alpha=0.3)

# Oklar ve etiketler
ax.annotate('', xy=(0.85, 0.7), xytext=(0.15, 0.55), 
            arrowprops=dict(arrowstyle='->', color=COLORS['domain_correct'], lw=2))
ax.annotate('', xy=(0.85, 0.25), xytext=(0.15, 0.45), 
            arrowprops=dict(arrowstyle='->', color=COLORS['wrong'], lw=2))

# Stage labels
for i, stage in enumerate(stages):
    ax.text(i, -0.05, stage, ha='center', va='top', fontsize=12, fontweight='bold', 
            color=COLORS['background'])

# Başlık ve legend
ax.set_xlim(-0.3, 3.3)
ax.set_ylim(-0.1, 1.05)
ax.axis('off')
ax.set_title('Hierarchical Pipeline Error Flow\n(Stage 1 → Stage 2 Error Propagation)', 
            fontsize=16, fontweight='bold', pad=20)

# Legend
legend_elements = [
    mpatches.Patch(color=COLORS['correct'], label=f'Success: {final_correct:,} ({final_correct_pct:.1f}%)'),
    mpatches.Patch(color=COLORS['wrong'], label=f'Stage 1 Error: {n_s1_wrong:,} ({s1_wrong_pct:.1f}%)'),
    mpatches.Patch(color=COLORS['mask_wrong'], label=f'Stage 2 Error: {n_s2_wrong:,} ({s2_wrong_pct:.1f}%)')
]
ax.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=11, 
         framealpha=0.9, bbox_to_anchor=(0.5, -0.02))

plt.tight_layout()
plt.savefig(os.path.join(viz_folder, 'error_flow_diagram.png'), 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"   ✅ Error flow diagram kaydedildi")

# =============================================
# 5️⃣ CONFIDENCE DISTRIBUTION PLOTS
# =============================================

print("\n5️⃣ Confidence Distribution Plots oluşturuluyor...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 5a. Domain Confidence - Correct vs Wrong
ax = axes[0, 0]
correct_confs = df_results[df_results['domain_correct'] == True]['domain_confidence'].values
wrong_confs = df_results[df_results['domain_correct'] == False]['domain_confidence'].values

ax.hist(correct_confs, bins=50, alpha=0.7, color=COLORS['correct'], 
        label=f'Correct ({len(correct_confs):,})', density=True, edgecolor='white')
ax.hist(wrong_confs, bins=50, alpha=0.7, color=COLORS['wrong'], 
        label=f'Wrong ({len(wrong_confs):,})', density=True, edgecolor='white')
ax.axvline(x=0.9, color=COLORS['background'], linestyle='--', linewidth=2, label='High Conf (0.9)')
ax.axvline(x=correct_confs.mean(), color=COLORS['correct'], linestyle=':', linewidth=2)
ax.axvline(x=wrong_confs.mean() if len(wrong_confs) > 0 else 0, color=COLORS['wrong'], linestyle=':', linewidth=2)
ax.set_xlabel('Domain Confidence', fontsize=12, fontweight='bold')
ax.set_ylabel('Density', fontsize=12, fontweight='bold')
ax.set_title('Stage 1: Domain Prediction Confidence\n(Correct vs Wrong)', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 5b. Mask Confidence (Domain Correct olanlar için)
ax = axes[0, 1]
mask_correct_confs = df_stage1_correct_stage2_correct['mask_confidence'].values
mask_wrong_confs = df_stage1_correct_stage2_wrong['mask_confidence'].values

if len(mask_correct_confs) > 0:
    ax.hist(mask_correct_confs, bins=50, alpha=0.7, color=COLORS['correct'], 
            label=f'Mask Correct ({len(mask_correct_confs):,})', density=True, edgecolor='white')
if len(mask_wrong_confs) > 0:
    ax.hist(mask_wrong_confs, bins=50, alpha=0.7, color=COLORS['mask_wrong'], 
            label=f'Mask Wrong ({len(mask_wrong_confs):,})', density=True, edgecolor='white')
ax.axvline(x=0.9, color=COLORS['background'], linestyle='--', linewidth=2, label='High Conf (0.9)')
ax.set_xlabel('Mask Confidence', fontsize=12, fontweight='bold')
ax.set_ylabel('Density', fontsize=12, fontweight='bold')
ax.set_title('Stage 2: Mask Prediction Confidence\n(Domain Correct Samples Only)', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 5c. Box Plot - Domain Confidence by Domain
ax = axes[1, 0]
domain_conf_data = []
domain_labels = []
for domain in DOMAIN_NAMES:
    df_dom = df_results[df_results['true_domain_name'] == domain]
    domain_conf_data.append(df_dom['domain_confidence'].values)
    domain_labels.append(domain)

bp = ax.boxplot(domain_conf_data, labels=domain_labels, patch_artist=True)
colors_box = [COLORS['domain_correct'], COLORS['correct'], COLORS['mask_wrong'], 
              COLORS['accent'], COLORS['neutral']]
for patch, color in zip(bp['boxes'], colors_box):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax.set_xlabel('Domain', fontsize=12, fontweight='bold')
ax.set_ylabel('Domain Confidence', fontsize=12, fontweight='bold')
ax.set_title('Domain Confidence Distribution by True Domain', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=45)

# 5d. Confidence vs Error Rate
ax = axes[1, 1]
conf_bins = np.linspace(0, 1, 11)
bin_centers = (conf_bins[:-1] + conf_bins[1:]) / 2
error_rates = []
sample_counts = []

for i in range(len(conf_bins) - 1):
    mask = (df_results['domain_confidence'] >= conf_bins[i]) & (df_results['domain_confidence'] < conf_bins[i+1])
    df_bin = df_results[mask]
    if len(df_bin) > 0:
        error_rate = 1 - df_bin['domain_correct'].mean()
        error_rates.append(error_rate)
        sample_counts.append(len(df_bin))
    else:
        error_rates.append(0)
        sample_counts.append(0)

# Bar plot
bars = ax.bar(bin_centers, error_rates, width=0.08, color=COLORS['wrong'], 
              edgecolor='white', alpha=0.85)

# Sample count as secondary y-axis
ax2 = ax.twinx()
ax2.plot(bin_centers, sample_counts, color=COLORS['domain_correct'], 
         marker='o', linewidth=2, label='Sample Count')
ax2.set_ylabel('Sample Count', fontsize=12, fontweight='bold', color=COLORS['domain_correct'])
ax2.tick_params(axis='y', labelcolor=COLORS['domain_correct'])

ax.set_xlabel('Confidence Bin', fontsize=12, fontweight='bold')
ax.set_ylabel('Error Rate', fontsize=12, fontweight='bold', color=COLORS['wrong'])
ax.tick_params(axis='y', labelcolor=COLORS['wrong'])
ax.set_title('Domain Error Rate vs Confidence\n(Lower confidence = Higher error)', fontsize=13, fontweight='bold')
ax.set_ylim([0, max(error_rates)*1.2 if error_rates else 1])
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(viz_folder, 'confidence_distributions.png'), 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"   ✅ Confidence distribution plots kaydedildi")

# =============================================
# 6️⃣ ERROR BREAKDOWN PIE CHART
# =============================================

print("\n6️⃣ Error Breakdown Pie Chart oluşturuluyor...")

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Sol: Genel breakdown
ax = axes[0]
labels = ['Stage1✓ Stage2✓\n(Full Success)', 'Stage1✓ Stage2✗\n(Mask Error)', 'Stage1✗\n(Domain Error)']
sizes = [len(df_stage1_correct_stage2_correct), len(df_stage1_correct_stage2_wrong), len(df_stage1_wrong)]
colors = [COLORS['correct'], COLORS['mask_wrong'], COLORS['wrong']]
explode = (0.02, 0.02, 0.05)

wedges, texts, autotexts = ax.pie(sizes, explode=explode, labels=labels, colors=colors,
                                   autopct='%1.1f%%', shadow=True, startangle=90,
                                   textprops={'fontsize': 11, 'fontweight': 'bold'},
                                   wedgeprops={'edgecolor': 'white', 'linewidth': 2})
ax.set_title(f'Pipeline Result Distribution\n(Total: {n_total:,} samples)', 
            fontsize=14, fontweight='bold')

# Sağ: Domain bazında error breakdown
ax = axes[1]
domain_errors = []
for domain in DOMAIN_NAMES:
    df_dom = df_results[df_results['true_domain_name'] == domain]
    domain_errors.append(len(df_dom[df_dom['domain_correct'] == False]))

bars = ax.barh(DOMAIN_NAMES, domain_errors, color=COLORS['wrong'], 
               edgecolor='white', alpha=0.85)
ax.set_xlabel('Number of Domain Errors', fontsize=12, fontweight='bold')
ax.set_ylabel('Domain', fontsize=12, fontweight='bold')
ax.set_title('Stage 1 Domain Errors by True Domain', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Bar labels
for bar, err in zip(bars, domain_errors):
    width = bar.get_width()
    ax.text(width + 10, bar.get_y() + bar.get_height()/2.,
            f'{err:,}',
            ha='left', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(viz_folder, 'error_breakdown.png'), 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"   ✅ Error breakdown pie chart kaydedildi")

print("\n" + "="*70)
print("✅ GÖRSELLEŞTİRME BÖLÜM 2 TAMAMLANDI!")
print("="*70)


In [ ]:
"""
💾 HÜCRE 25 - SONUÇLARI DIŞA AKTAR VE ÖZET RAPOR
CSV, JSON ve HTML formatlarında sonuçları kaydet
"""

from datetime import datetime
import json

print("\n" + "="*70)
print("💾 SONUÇLARI DIŞA AKTAR")
print("="*70)

# =============================================
# 8️⃣ CSV EXPORT
# =============================================

print("\n8️⃣ Detaylı sonuçlar CSV'ye kaydediliyor...")

# Main results CSV
csv_path = os.path.join(output_folder, 'detailed_results.csv')
df_results.to_csv(csv_path, index=False)
print(f"   ✅ Detaylı sonuçlar: {csv_path}")

# Domain analysis CSV
domain_csv_path = os.path.join(output_folder, 'domain_analysis.csv')
df_domain_analysis.to_csv(domain_csv_path, index=False)
print(f"   ✅ Domain analizi: {domain_csv_path}")

# Error pairs CSV
error_pairs_path = os.path.join(output_folder, 'domain_confusion_pairs.csv')
domain_confusion_pairs.to_csv(error_pairs_path, index=False)
print(f"   ✅ Domain karışıklık çiftleri: {error_pairs_path}")

# =============================================
# 9️⃣ JSON SUMMARY EXPORT
# =============================================

print("\n9️⃣ Özet metrikler JSON'a kaydediliyor...")

summary_json = {
    'metadata': {
        'generated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'notebook': NOTEBOOK_NAME,
        'total_validation_samples': int(total_samples)
    },
    'stage1_domain_classification': {
        'accuracy': float(domain_accuracy),
        'correct_count': int(domain_correct_count),
        'wrong_count': int(total_samples - domain_correct_count),
        'per_domain_accuracy': {
            row['domain']: float(row['stage1_accuracy']) 
            for _, row in df_domain_analysis.iterrows()
        }
    },
    'stage2_mask_detection': {
        'accuracy_on_correct_domain': float(mask_accuracy_on_correct),
        'correct_count': int(mask_correct_count),
        'wrong_count': int(len(df_domain_correct) - mask_correct_count) if len(df_domain_correct) > 0 else 0,
        'per_domain_accuracy': {
            row['domain']: float(row['stage2_accuracy']) 
            for _, row in df_domain_analysis.iterrows()
        }
    },
    'full_pipeline': {
        'accuracy': float(pipeline_accuracy),
        'correct_count': int(pipeline_correct_count),
        'wrong_count': int(total_samples - pipeline_correct_count),
        'per_domain_accuracy': {
            row['domain']: float(row['pipeline_accuracy']) 
            for _, row in df_domain_analysis.iterrows()
        }
    },
    'error_breakdown': {
        'stage1_correct_stage2_correct': int(len(df_stage1_correct_stage2_correct)),
        'stage1_correct_stage2_wrong': int(len(df_stage1_correct_stage2_wrong)),
        'stage1_wrong': int(len(df_stage1_wrong))
    },
    'confidence_analysis': {
        'avg_confidence_correct_domain': float(avg_conf_correct),
        'avg_confidence_wrong_domain': float(avg_conf_wrong),
        'low_confidence_threshold': 0.7,
        'low_confidence_samples': int(len(df_low_conf)),
        'high_confidence_threshold': 0.9,
        'high_confidence_samples': int(len(df_high_conf))
    },
    'top_domain_confusions': [
        {
            'true_domain': row['true_domain_name'],
            'pred_domain': row['final_pred_domain_name'],
            'count': int(row['count'])
        }
        for _, row in domain_confusion_pairs.head(5).iterrows()
    ]
}

json_path = os.path.join(output_folder, 'summary_metrics.json')
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(summary_json, f, indent=2, ensure_ascii=False)
print(f"   ✅ Özet metrikler: {json_path}")

# =============================================
# 🔟 FINAL SUMMARY DISPLAY
# =============================================

print("\n" + "="*70)
print("📊 KAPSAMLI HİYERARŞİK VALİDASYON TEST SONUÇLARI")
print("="*70)

print(f"""
╔══════════════════════════════════════════════════════════════════════╗
║                         📋 TEST ÖZETİ                                 ║
╚══════════════════════════════════════════════════════════════════════╝

   📅 Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
   📊 Toplam Test Örnekleri: {total_samples:,}

╔══════════════════════════════════════════════════════════════════════╗
║                    🎯 STAGE 1 - DOMAIN CLASSIFICATION                 ║
╚══════════════════════════════════════════════════════════════════════╝

   ✅ Accuracy: {domain_accuracy*100:.2f}%
   ✅ Doğru:    {domain_correct_count:,} örnek
   ❌ Yanlış:   {total_samples - domain_correct_count:,} örnek

╔══════════════════════════════════════════════════════════════════════╗
║                    🎭 STAGE 2 - MASK DETECTION                        ║
║                  (Sadece Domain Doğru Olanlar İçin)                   ║
╚══════════════════════════════════════════════════════════════════════╝

   ✅ Accuracy: {mask_accuracy_on_correct*100:.2f}%
   ✅ Doğru:    {mask_correct_count:,} örnek
   ❌ Yanlış:   {len(df_domain_correct) - mask_correct_count:,} örnek

╔══════════════════════════════════════════════════════════════════════╗
║                    🔗 FULL PIPELINE PERFORMANCE                       ║
╚══════════════════════════════════════════════════════════════════════╝

   ✅ Pipeline Accuracy: {pipeline_accuracy*100:.2f}%
   ✅ Tam Başarı:  {pipeline_correct_count:,} örnek ({pipeline_accuracy*100:.1f}%)
   ⚠️ Stage 1 Hatası: {len(df_stage1_wrong):,} örnek ({len(df_stage1_wrong)/total_samples*100:.1f}%)
   ⚠️ Stage 2 Hatası: {len(df_stage1_correct_stage2_wrong):,} örnek ({len(df_stage1_correct_stage2_wrong)/total_samples*100:.1f}%)

╔══════════════════════════════════════════════════════════════════════╗
║                         📊 DOMAIN BAZINDA                             ║
╚══════════════════════════════════════════════════════════════════════╝
""")

print(f"   {'Domain':<15} {'S1 Acc':>10} {'S2 Acc':>10} {'Pipeline':>10}")
print(f"   {'-'*50}")
for _, row in df_domain_analysis.iterrows():
    print(f"   {row['domain']:<15} {row['stage1_accuracy']*100:>9.1f}% {row['stage2_accuracy']*100:>9.1f}% {row['pipeline_accuracy']*100:>9.1f}%")

print(f"""
╔══════════════════════════════════════════════════════════════════════╗
║                         📂 OLUŞTURULAN DOSYALAR                       ║
╚══════════════════════════════════════════════════════════════════════╝

   📁 {output_folder}/
      ├── 📄 detailed_results.csv          (Her örnek için detaylı sonuçlar)
      ├── 📄 domain_analysis.csv           (Domain bazında analiz)
      ├── 📄 domain_confusion_pairs.csv    (Domain karışıklık çiftleri)
      ├── 📄 summary_metrics.json          (Özet metrikler)
      │
      └── 📁 visualizations/
          ├── 📊 stage1_domain_confusion_matrix.png
          ├── 📊 stage2_mask_confusion_matrices.png
          ├── 📊 per_domain_accuracy.png
          ├── 📊 error_flow_diagram.png
          ├── 📊 confidence_distributions.png
          ├── 📊 error_breakdown.png
          ├── 🖼️ misclassified_stage1_errors.png
          ├── 🖼️ misclassified_stage2_errors.png
          └── 🖼️ success_samples.png

╔══════════════════════════════════════════════════════════════════════╗
║                         💡 ÖNERİLER                                   ║
╚══════════════════════════════════════════════════════════════════════╝
""")

# Recommendations based on results
if domain_accuracy < 0.85:
    print(f"   ⚠️ Stage 1 accuracy düşük ({domain_accuracy*100:.1f}% < 85%)")
    print(f"      → Domain classifier modelini iyileştir")
    print(f"      → Daha fazla epoch veya data augmentation dene")

if mask_accuracy_on_correct < 0.75:
    print(f"   ⚠️ Stage 2 accuracy düşük ({mask_accuracy_on_correct*100:.1f}% < 75%)")
    print(f"      → Domain-specific mask detector'ları iyileştir")
    print(f"      → Sınıf dengesizliği için class weighting dene")

if len(df_stage1_wrong) > total_samples * 0.2:
    print(f"   🚨 Yüksek Stage 1 hata oranı!")
    print(f"      → {len(df_stage1_wrong):,} örnek yanlış domain'e gitti")
    print(f"      → En çok karışan domain çiftlerini incele")

if avg_conf_wrong > 0.7:
    print(f"   ⚠️ Yanlış tahminlerin confidence'ı yüksek ({avg_conf_wrong:.2f})")
    print(f"      → Model calibration gerekebilir")
    print(f"      → Label smoothing veya temperature scaling dene")

print(f"""
══════════════════════════════════════════════════════════════════════
                    ✅ TEST TAMAMLANDI!
══════════════════════════════════════════════════════════════════════
""")
